# Setting up the environment

We'll load the needed libraries:


In [8]:
# install.packages("styler")
# library(styler)
# styler::style_file(path = "Calculations.Rmd")

ERROR: Error in library(styler): there is no package called ‘styler’


In [3]:
# packrat::status()

Up to date.



In [1]:
options(repr.matrix.max.rows = 100, repr.matrix.max.cols = 300)
options(repr.plot.width = 20, repr.plot.height = 15)
options(width = 300)

numcores <- 56

library(tidyverse)
library(data.table)
library(fst)
library(comorbidity)
library(reshape)
library(dtplyr)
library(haven)
library(vroom)
library(dplyr)
`%!in%` <- Negate(`%in%`)

setDTthreads(numcores)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


This is {comorbidity} version 1.0.3.

A lot has changed since the pre-1.0.0 release on CRAN, please check-out breaking changes here:

-> https://ellessenne.githu

# Codes

First, we will add codes from ICD and Medicare:primary_care_specialty_codes

In [2]:
# diagnosis codes

office_visit_codes <- c(
  "99201", "99202", "99203", "99204", "99205", "99211", "99212", "99213", "99214",
  "99215"
)

IHD_icd_9_codes <- c(410, 411, 412, 413, 414)
IHD_icd_10_codes <- c("I20", "I21", "I22", "I23", "I24", "I25")

non_us_state_codes <- c(40, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 97, 98, 99)

primary_care_specialty_codes <- c("01", "08", "11", "38")

# http://www.icd9data.com/2015/Volume1/390-459/401-405/default.htm
# https://www.icd10data.com/ICD10CM/Codes/I00-I99/I10-I16
hypertension_icd_9_codes <- c("401", "402", "403", "404", "405")
hypertension_icd_10_codes <- c("I10", "I11", "I12", "I13", "I15", "I16")

# http://www.icd9data.com/2014/Volume1/290-319/295-299/296/default.htm
# https://www.icd10data.com/ICD10CM/Codes/F01-F99/F30-F39
depression_icd_9_codes <- c("2962", "2963")
depression_icd_10_codes <- c("F32", "F33")

# http://www.icd9data.com/2015/Volume1/240-279/249-259/default.htm
# https://www.icd10data.com/ICD10CM/Codes/E00-E89/E08-E13
diabetes_icd_9_codes <- c("250")
diabetes_icd_10_codes <- c("E08", "E09", "E10", "E11", "E13")

# http://www.icd9data.com/2014/Volume1/710-739/710-719/714/default.htm
# https://www.icd10data.com/ICD10CM/Codes/M00-M99/M05-M14
arthritis_icd_9_codes <- c("714")
arthritis_icd_10_codes <- c("M05", "M06", "M07", "M08", "M09", "M10", "M11", "M12", "M13", "M14")


race_codes <- data.frame(
  race_code = seq(0, 6),
  race = c("Unknown", "White", "Black", "Other", "Asian", "Hispanic", "North American Native")
)

sex_codes <- data.frame(
  sex_code = seq(0, 2),
  sex = c("Unknown", "Male", "Female")
)

# Patient level calculations

## Yearly Calculators

These are the main functions that calculate yearly expenditures for patients and their corresponding physicians.\

### Fixing the MBSF data

In [10]:
mbsf_2013 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2013.csv", num_threads = numcores) %>% as.data.table()
mbsf_2014 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2014.csv", num_threads = numcores) %>% as.data.table()
mbsf_2015 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/den_saf_lds_5_2015.csv", num_threads = numcores) %>% as.data.table()
mbsf_2016 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2016.csv", num_threads = numcores) %>% as.data.table()
mbsf_2017 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2017.csv", num_threads = numcores) %>% as.data.table()
mbsf_2018 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2018.csv", num_threads = numcores) %>% as.data.table()
mbsf_2019 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2019.csv", num_threads = numcores) %>% as.data.table()
mbsf_2020 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/MBSF/mbsf_lds_5_2020.csv", num_threads = numcores) %>% as.data.table()

New names:
• `1` -> `1...5`
• `1` -> `1...7`
• `0` -> `0...8`
• `0` -> `0...9`
• `0` -> `0...11`
• `0` -> `0...12`
• `3` -> `3...13`
• `3` -> `3...14`
• `3` -> `3...15`
• `3` -> `3...16`
• `3` -> `3...17`
• `3` -> `3...18`
• `3` -> `3...19`
• `3` -> `3...20`
• `3` -> `3...21`
• `3` -> `3...22`
• `3` -> `3...23`
• `3` -> `3...24`
• `C` -> `C...25`
• `C` -> `C...26`
• `C` -> `C...27`
• `C` -> `C...28`
• `C` -> `C...29`
• `C` -> `C...30`
• `C` -> `C...31`
• `C` -> `C...32`
• `C` -> `C...33`
• `C` -> `C...34`
• `C` -> `C...35`
• `C` -> `C...36`
• `12` -> `12...37`
• `12` -> `12...38`
• `12` -> `12...39`
• `0` -> `0...40`
• `` -> `...41`
• `` -> `...42`
Rows: 2758309 Columns: 43
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (29): 000000000, 05, 400, 0...9, 3...13, 3...14, 3...15, 3...16, 3...17, 3...18, 3...19, 3...20, 3...21, 3...22, 3...23, 3...24, C...25, C...26, C...27, C...

In [11]:
mbsf_colnames_2013_2015 <- c("DESY_SORT_KEY", "STATE_CODE", "COUNTY_CODE", "SEX_CODE", "RACE_CODE", "AGE", "ORIG_REASON_FOR_ENTITLEMENT", "CURR_REASON_FOR_ENTITLEMENT", "ESRD_INDICATOR", "MEDICARE_STATUS_CD", "PART_A_TERMINATION_CODE", "PART_B_TERMINATION_CODE", "ENTITLEMENT_BUY_IN_IND01", "ENTITLEMENT_BUY_IN_IND02", "ENTITLEMENT_BUY_IN_IND03", "ENTITLEMENT_BUY_IN_IND04", "ENTITLEMENT_BUY_IN_IND05", "ENTITLEMENT_BUY_IN_IND06", "ENTITLEMENT_BUY_IN_IND07", "ENTITLEMENT_BUY_IN_IND08", "ENTITLEMENT_BUY_IN_IND09", "ENTITLEMENT_BUY_IN_IND10", "ENTITLEMENT_BUY_IN_IND11", "ENTITLEMENT_BUY_IN_IND12", "HMO_INDICATOR01", "HMO_INDICATOR02", "HMO_INDICATOR03", "HMO_INDICATOR04", "HMO_INDICATOR05", "HMO_INDICATOR06", "HMO_INDICATOR07", "HMO_INDICATOR08", "HMO_INDICATOR09", "HMO_INDICATOR10", "HMO_INDICATOR11", "HMO_INDICATOR12", "HI_COVERAGE", "SMI_COVERAGE", "HMO_COVERAGE", "STATE_BUY_IN_COVERAGE", "VALID_DATE_OF_DEATH_SWITCH", "DATE_OF_DEATH", "REFERENCE_YEAR")

mbsf_colnames_2016_2020 <- c("DESY_SORT_KEY", "REFERENCE_YEAR", "SAMPLE_GROUP", "STATE_CODE", "COUNTY_CODE", "STATE_CNTY_FIPS_CD_01", "STATE_CNTY_FIPS_CD_02", "STATE_CNTY_FIPS_CD_03", "STATE_CNTY_FIPS_CD_04", "STATE_CNTY_FIPS_CD_05", "STATE_CNTY_FIPS_CD_06", "STATE_CNTY_FIPS_CD_07", "STATE_CNTY_FIPS_CD_08", "STATE_CNTY_FIPS_CD_09", "STATE_CNTY_FIPS_CD_10", "STATE_CNTY_FIPS_CD_11", "STATE_CNTY_FIPS_CD_12", "SEX_CODE", "RACE_CODE", "AGE", "ORIG_REASON_FOR_ENTITLEMENT", "CURR_REASON_FOR_ENTITLEMENT", "ESRD_INDICATOR", "MDCR_STATUS_CODE_01", "MDCR_STATUS_CODE_02", "MDCR_STATUS_CODE_03", "MDCR_STATUS_CODE_04", "MDCR_STATUS_CODE_05", "MDCR_STATUS_CODE_06", "MDCR_STATUS_CODE_07", "MDCR_STATUS_CODE_08", "MDCR_STATUS_CODE_09", "MDCR_STATUS_CODE_10", "MDCR_STATUS_CODE_11", "MDCR_STATUS_CODE_12", "PART_A_TERMINATION_CODE", "PART_B_TERMINATION_CODE", "ENTITLEMENT_BUY_IN_IND01", "ENTITLEMENT_BUY_IN_IND02", "ENTITLEMENT_BUY_IN_IND03", "ENTITLEMENT_BUY_IN_IND04", "ENTITLEMENT_BUY_IN_IND05", "ENTITLEMENT_BUY_IN_IND06", "ENTITLEMENT_BUY_IN_IND07", "ENTITLEMENT_BUY_IN_IND08", "ENTITLEMENT_BUY_IN_IND09", "ENTITLEMENT_BUY_IN_IND10", "ENTITLEMENT_BUY_IN_IND11", "ENTITLEMENT_BUY_IN_IND12", "HMO_INDICATOR01", "HMO_INDICATOR02", "HMO_INDICATOR03", "HMO_INDICATOR04", "HMO_INDICATOR05", "HMO_INDICATOR06", "HMO_INDICATOR07", "HMO_INDICATOR08", "HMO_INDICATOR09", "HMO_INDICATOR10", "HMO_INDICATOR11", "HMO_INDICATOR12", "HI_COVERAGE", "SMI_COVERAGE", "HMO_COVERAGE", "STATE_BUY_IN_COVERAGE", "VALID_DATE_OF_DEATH_SWITCH", "DATE_OF_DEATH", "DUAL_STUS_CD_01", "DUAL_STUS_CD_02", "DUAL_STUS_CD_03", "DUAL_STUS_CD_04", "DUAL_STUS_CD_05", "DUAL_STUS_CD_06", "DUAL_STUS_CD_07", "DUAL_STUS_CD_08", "DUAL_STUS_CD_09", "DUAL_STUS_CD_10", "DUAL_STUS_CD_11", "DUAL_STUS_CD_12")


colnames(mbsf_2013) <- mbsf_colnames_2013_2015
colnames(mbsf_2014) <- mbsf_colnames_2013_2015
colnames(mbsf_2015) <- mbsf_colnames_2013_2015
colnames(mbsf_2016) <- mbsf_colnames_2016_2020
colnames(mbsf_2017) <- mbsf_colnames_2016_2020
colnames(mbsf_2018) <- mbsf_colnames_2016_2020
colnames(mbsf_2019) <- mbsf_colnames_2016_2020
colnames(mbsf_2020) <- mbsf_colnames_2016_2020

In [35]:
mbsf_needed_cols <- c(
  "DESY_SORT_KEY",
  "REFERENCE_YEAR",
  "STATE_CODE",
  "COUNTY_CODE",
  "SEX_CODE",
  "RACE_CODE",
  "AGE",
  "ORIG_REASON_FOR_ENTITLEMENT",
  "CURR_REASON_FOR_ENTITLEMENT",
  "ENTITLEMENT_BUY_IN_IND01",
  "ENTITLEMENT_BUY_IN_IND02",
  "ENTITLEMENT_BUY_IN_IND03",
  "ENTITLEMENT_BUY_IN_IND04",
  "ENTITLEMENT_BUY_IN_IND05",
  "ENTITLEMENT_BUY_IN_IND06",
  "ENTITLEMENT_BUY_IN_IND07",
  "ENTITLEMENT_BUY_IN_IND08",
  "ENTITLEMENT_BUY_IN_IND09",
  "ENTITLEMENT_BUY_IN_IND10",
  "ENTITLEMENT_BUY_IN_IND11",
  "ENTITLEMENT_BUY_IN_IND12",
  "HMO_INDICATOR01",
  "HMO_INDICATOR02",
  "HMO_INDICATOR03",
  "HMO_INDICATOR04",
  "HMO_INDICATOR05",
  "HMO_INDICATOR06",
  "HMO_INDICATOR07",
  "HMO_INDICATOR08",
  "HMO_INDICATOR09",
  "HMO_INDICATOR10",
  "HMO_INDICATOR11",
  "HMO_INDICATOR12",
  "VALID_DATE_OF_DEATH_SWITCH",
  "DATE_OF_DEATH"
)

mbsf_data <- list(mbsf_2013, mbsf_2014, mbsf_2015, mbsf_2016, mbsf_2017, mbsf_2018, mbsf_2019, mbsf_2020)

for (a in 1:length(mbsf_data)) {
  mbsf_data[[a]] <- mbsf_data[[a]][, ..mbsf_needed_cols]
}

# rename last n columns in a dataset
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}


for (a in 1:length(mbsf_data)) {
  mbsf_data[[a]][, year := 2012 + a]
}



mbsf_data <- reduce(mbsf_data, function(x, y) {
  rbind(x, y)
}) %>% as.data.table()

In [37]:
head(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,49,801,2,1,71,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,33,400,2,1,75,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013
000000000,13,10,510,1,1,70,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013


In [38]:
mbsf_data_death_collapsed <- mbsf_data[!is.na(DATE_OF_DEATH)]

mbsf_data_death_collapsed$date_died <- mbsf_data_death_collapsed$DATE_OF_DEATH

mbsf_data <-
  left_join(mbsf_data,
    mbsf_data_death_collapsed[, .(
      DESY_SORT_KEY,
      date_died
    )],
    by = ("DESY_SORT_KEY")
  ) %>% as.data.table()

mbsf_data_death_collapsed <- mbsf_data[!is.na(VALID_DATE_OF_DEATH_SWITCH)]

mbsf_data_death_collapsed$date_died_valid <- mbsf_data_death_collapsed$VALID_DATE_OF_DEATH_SWITCH

mbsf_data <-
  left_join(mbsf_data,
    mbsf_data_death_collapsed[, .(
      DESY_SORT_KEY,
      date_died_valid
    )],
    by = ("DESY_SORT_KEY")
  ) %>% as.data.table()


head(mbsf_data)

DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year,date_died
<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017
000000000,13,33,420,2,1,93,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627


In [43]:
write_fst(mbsf_data, "/work/postresearch/Shared/Projects/Farbod/CaseMix/mbsf_data_long.fst")

### Read data



In [190]:
carrier_data_all_years <- read_fst(
  "/work/postresearch/Shared/Projects/Farbod/carrier_data_all_years.fst",
  as.data.table = T
)
# mbsf_data = read_fst(
#  "/work/postresearch/Shared/Projects/Farbod/CaseMix/mbsf_data_long.fst", as.data.table = T)
# mbsf_data[,DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]

In [191]:
head(carrier_data_all_years)
head(mbsf_data)

DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>
100000015,2,1,20130425,22,94375,9,496,15.26,1073503884,29,22,2013-04-25,2013,2013-04
100000015,2,2,20130425,22,94726,9,496,13.54,1073503884,29,22,2013-04-25,2013,2013-04
100000015,2,3,20130425,22,94729,9,496,9.95,1073503884,29,22,2013-04-25,2013,2013-04
100000015,3,1,20130528,11,99214,9,41400,114.64,1285600932,11,22,2013-05-28,2013,2013-05
100000015,3,2,20130528,11,93000,9,41400,20.08,1285600932,11,22,2013-05-28,2013,2013-05
100000015,4,1,20130719,22,99213,9,496,51.76,1659344091,29,22,2013-07-19,2013,2013-07


DESY_SORT_KEY,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,year,date_died,date_died_valid
<int>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627,V
0,13,45,910,2,1,75,0,0,C,C,C,C,C,C,C,C,C,C,C,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20150627,V
0,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V
0,13,22,170,2,1,71,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,2013,20131017,V


### Loading sample data (for pc)

In [4]:
sample_data <- readRDS(file = "sample_data.RDS")

In [5]:
carrier_data_all_years <- sample_data[[1]]
outpatient_data_all_years <- sample_data[[2]]
inpatient_data_all_years <- sample_data[[3]]
mbsf_data <- read_fst("mbsf_data_long.fst", as.data.table = T)
revenue_center_outpatient_all_years <- sample_data[[5]]
outpatient_and_revenue_center_data <- sample_data[[6]]

### Patient yearly expenditures and use of services carrier

I will first create a function that adds conditions of interest to the data.


#### Finding conditions for each claim line

In [46]:
yearly_calculator_patient_conditions <- function(data) {

  # requirements
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  require(lubridate)

  data %>%
    mutate(
      is_office_visit = HCPCS_CD %in% office_visit_codes,
      is_by_primary_care_physician = PRVDR_SPCLTY %in% primary_care_specialty_codes,
      is_hypertension = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes, NA
        )
      ),
      is_arthritis = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes, NA
        )
      ),
      is_IHD = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% IHD_icd_9_codes, NA
        )
      ),
      is_diabetes = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes, NA
        )
      ),
      is_depression = if_else(
        LINE_ICD_DGNS_VRSN_CD == 0,
        substr(LINE_ICD_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
        if_else(
          LINE_ICD_DGNS_VRSN_CD == 9,
          substr(LINE_ICD_DGNS_CD, 0, 4) %in% depression_icd_9_codes, NA
        )
      )
    ) %>%
    as.data.table()
}

yearly_patient_conditions_carrier <- yearly_calculator_patient_conditions(carrier_data_all_years)
head(yearly_patient_conditions_carrier)

DESY_SORT_KEY,CLAIM_NO,LINE_NUM,CLM_THRU_DT,LINE_PLACE_OF_SRVC_CD,HCPCS_CD,LINE_ICD_DGNS_VRSN_CD,LINE_ICD_DGNS_CD,LINE_ALOWD_CHRG_AMT,PRF_PHYSN_NPI,PRVDR_SPCLTY,PRVDR_STATE_CD,date,year,month_year,is_office_visit,is_by_primary_care_physician,is_hypertension,is_arthritis,is_IHD,is_diabetes,is_depression
<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<chr>,<int>,<date>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100000015,2,1,20130425,22,94375,9,496,15.26,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2,2,20130425,22,94726,9,496,13.54,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2,3,20130425,22,94729,9,496,9.95,1073503884,29,22,2013-04-25,2013,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,3,1,20130528,11,99214,9,41400,114.64,1285600932,11,22,2013-05-28,2013,2013-05,TRUE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,3,2,20130528,11,93000,9,41400,20.08,1285600932,11,22,2013-05-28,2013,2013-05,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,4,1,20130719,22,99213,9,496,51.76,1659344091,29,22,2013-07-19,2013,2013-07,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


#### Summarizing patient data
I will now summarise the data for each patient.


In [ ]:
summarise_carrier <- function(data, time_frame = 365) {
  data %>%
    group_by(DESY_SORT_KEY, year) %>%
    summarise(
      # tot_allowed_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT),

      # office_visit_count = sum(na.rm = T, is_office_visit),

      # office_visit_cost_carrier = sum(na.rm = T, LINE_ALOWD_CHRG_AMT * is_office_visit),
      distinct_clinicians = length(unique(PRF_PHYSN_NPI)),
      distinct_primary_care_physicians = length(.[is_by_primary_care_physician, unique(PRF_PHYSN_NPI)]),
      hypertension = sum(is_hypertension, na.rm = T) > 0,
      arthritis = sum(is_arthritis, na.rm = T) > 0,
      IHD = sum(is_IHD, na.rm = T) > 0,
      diabetes = sum(is_diabetes, na.rm = T) > 0,
      depression = sum(is_depression, na.rm = T) > 0,
      icd_9_pure = ifelse(prod(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, F, T),
      icd_10_pure = ifelse(sum(LINE_ICD_DGNS_VRSN_CD, na.rm = T) == 0, T, F),
    ) %>%
    as.data.table()
}


summary_patient_by_year <- summarise_carrier(yearly_patient_conditions_carrier)
head(summary_patient_by_year)

`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


In [184]:
add_patient_characteristics <- function(mbsf_data, summary_data) {
  mbsf_data <- unique(mbsf_data)
  require(dtplyr)
  require(lubridate)
  require(tidyverse)
  data <- left_join(summary_data, mbsf_data, by = c("DESY_SORT_KEY", "year")) %>% as.data.table()

  data %>%
    mutate(
      year_of_death = substr(date_died, 0, 4)
    ) %>%
    as.data.table()
}

summary_with_patient_characteristics <- add_patient_characteristics(mbsf_data, summary_patient_by_year)
head(summary_with_patient_characteristics)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA
100000015,2018,16,2,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2018,22,160,1,1,81,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA


In [188]:
write.fst(summary_with_patient_characteristics, "summary_with_patient_characteristics_before_join.fst")
write.fst(summary_patient_by_year, "summary_patient_by_year_before_join.fst")

### Most common physicians for each patient

Now, we will find most common physicians and cardiologists for each patient.


In [ ]:
summary_with_patient_characteristics <- read.fst("summary_with_patient_characteristics_before_join.fst", as.data.table = T)
summary_patient_by_year <- read.fst("summary_patient_by_year_before_join.fst", as.data.table = T)

In [192]:
# adding most common physicians
add_patient_NPI <- function(data, summary_data, time_frame = 365) {
  comorbidity_and_phys_data <-
    inner_join(data, summary_data[, c(
      "DESY_SORT_KEY", "year",
      "icd_9_pure",
      "icd_10_pure"
    )], by = c("DESY_SORT_KEY", "year")) %>%
    as.data.table()

  patient_NPI_count_finder <- function(data) {
    result <- data %>%
      mutate(is_office_visit = HCPCS_CD %in% office_visit_codes) %>%
      group_by(DESY_SORT_KEY, year, PRF_PHYSN_NPI) %>%
      summarise(n = sum(is_office_visit, na.rm = T)) %>%
      filter(n > 0) %>%
      arrange(.by_group = T, desc(n))
  }

  patient_NPI_counts <- patient_NPI_count_finder(comorbidity_and_phys_data)

  patient_NPI_counts <- left_join(patient_NPI_counts,
    distinct(data[, .(PRF_PHYSN_NPI, PRVDR_SPCLTY)]),
    by = "PRF_PHYSN_NPI"
  )

  find_most_common <- function(data) {
    data %>%
      group_by(DESY_SORT_KEY, year) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }

  find_most_common_by_specialty <- function(data, specialty_code) {
    data %>%
      filter(PRVDR_SPCLTY %in% specialty_code) %>%
      group_by(DESY_SORT_KEY, year) %>%
      arrange(.by_group = T, desc(n)) %>%
      slice(1) %>%
      as.data.table()
  }

  most_common_physician <- find_most_common(patient_NPI_counts)
  most_common_primary_care_physician <- find_most_common_by_specialty(patient_NPI_counts,
    specialty_code = c("01", "08", "11", "38")
  )
  most_common_physician <- data.frame(most_common_physician) %>%
    rename_with(~ paste0("most_common_physician_", .x))
  most_common_primary_care_physician <- data.frame(most_common_primary_care_physician) %>%
    rename_with(~ paste0("most_common_primary_care_physician_", .x))

  summary_data <- left_join(
    summary_data,
    most_common_physician,
    by = c("DESY_SORT_KEY" = "most_common_physician_DESY_SORT_KEY", "year" = "most_common_physician_year")
  )
  summary_data <- left_join(
    summary_data,
    most_common_primary_care_physician,
    by = c("DESY_SORT_KEY" = "most_common_primary_care_physician_DESY_SORT_KEY", "year" = "most_common_primary_care_physician_year")
  ) %>%
    as.data.table()
}

summary_with_npi <- add_patient_NPI(data = carrier_data_all_years, summary_data = summary_with_patient_characteristics)
head(summary_with_npi)

`summarise()` has grouped output by 'DESY_SORT_KEY', 'year'. You can override using the `.groups` argument.


ERROR: Error: cannot allocate vector of size 5.5 Gb


In [ ]:
write.fst(summary_with_npi, "summary_with_npi_before_join.fst")

## Physician integration status

Here, I will find which physicians are integrated.


### A method of finding codes that are not exclusive to hospitals or non-hospital places
We can exclude these HCPCS codes and only include codes that are not exclusive to hospitals.

In [10]:
exclusive_hospital_code_finder <- function(data,
                                           threshold = 0.05,
                                           integrated_place_of_service_codes = c("19", "22"),
                                           all_place_of_service_codes = c("11", "19", "22")) {
  require(dtplyr)
  require(tidyverse)

  result <- data %>%
    filter(LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes) %>%
    group_by(HCPCS_CD) %>%
    summarise(prp_in_facility = nrow(.[LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes]) / n()) %>%
    as.data.table()

  exclusive_codes <- result[prp_in_facility < threshold | prp_in_facility > (1 - threshold), HCPCS_CD]


  return(exclusive_codes)
}

exclusive_hospital_codes <- exclusive_hospital_code_finder(carrier_data_all_years)

### A function to find integrated docs

In [12]:
# calculate and add physician integration data
# this only uses visits to see if a physician is integrated or not (codde list)

physician_integration_finder <- function(data,
                                         integrated_place_of_service_codes = c("19", "22"),
                                         all_place_of_service_codes = c("11", "19", "22"),
                                         # integration_threshold = 0.5,
                                         office_code_list = c(
                                           "99201",
                                           "99202",
                                           "99203",
                                           "99204",
                                           "99205",
                                           "99211",
                                           "99212",
                                           "99213",
                                           "99214",
                                           "99215"
                                         ),
                                         exclusive_hospital_codes) {
  require(dtplyr)
  require(tidyverse)

  # data = subset(data, HCPCS_CD %in% code_list)
  result <- data %>%
    mutate(
      is_facility = LINE_PLACE_OF_SRVC_CD %in% integrated_place_of_service_codes,
      is_all = LINE_PLACE_OF_SRVC_CD %in% all_place_of_service_codes,
      is_office_visit = HCPCS_CD %in% office_code_list,
      has_non_exclusive_code = HCPCS_CD %!in% exclusive_hospital_codes
    ) %>%
    group_by(PRF_PHYSN_NPI, year) %>%
    summarise(
      in_facility_visits_count = sum(is_facility * is_office_visit, na.rm = T),
      in_all_visits_count = sum(is_all * is_office_visit, na.rm = T),
      in_facility_non_exclusive_HCPCS_count = sum(is_facility * has_non_exclusive_code, na.rm = T),
      in_all_non_exclusive_HCPCS_count = sum(is_all * has_non_exclusive_code, na.rm = T),
      in_facility_count = sum(is_facility, na.rm = T),
      in_all_count = sum(is_all, na.rm = T)
    ) %>%
    mutate(
      in_facility_visits_prp = in_facility_visits_count / in_all_visits_count,
      in_facility_non_exclusive_HCPCS_prp = in_facility_non_exclusive_HCPCS_count / in_all_non_exclusive_HCPCS_count,
      in_facility_prp = in_facility_count / in_all_count
    ) %>%
    as.data.table()
}

physician_integration_stats <- physician_integration_finder(carrier_data_all_years, exclusive_hospital_codes = exclusive_hospital_codes)

`summarise()` has grouped output by 'PRF_PHYSN_NPI'. You can override using the `.groups` argument.


In [13]:
tail(physician_integration_stats)

PRF_PHYSN_NPI,year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp
<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1992997522,2013,2,2,10,10,10,10,1,1,1
1992998157,2013,0,0,0,0,0,0,NaN,NaN,NaN
1992998207,2013,0,0,0,0,0,5,NaN,NaN,0
1992999122,2013,0,8,0,8,0,8,0,0,0
9999999991,2013,0,1,0,2,0,2,0,0,0
9999999992,2013,0,3,0,16,0,71,0,0,0


### Add integration status of physicians
This function will add the integration status of most common physicians to each patient's summary data.

In [12]:
# summary_with_npi <- read.fst("./summary_with_npi_before_join.fst", as.data.table = T)

physician_integration_stats <- read.fst("./physician_integration_stats_before_join.fst", as.data.table = T)

In [ ]:
# rename columns
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}
add_integration_status <- function(data, physician_integration_stats) {
  data_selected <- data[, c(
    "DESY_SORT_KEY",
    "year",
    "most_common_physician_PRF_PHYSN_NPI",
    "most_common_primary_care_physician_PRF_PHYSN_NPI"
  )]

  most_common_physician <- left_join(
    data_selected,
    physician_integration_stats,
    by = c(
      "most_common_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year"
    )
  ) %>% as.data.table()

  most_common_physician <- most_common_physician[, -c("most_common_primary_care_physician_PRF_PHYSN_NPI")]

  rename_last(
    most_common_physician,
    ncol(physician_integration_stats) - 2,
    paste("most_common_physician_", colnames(physician_integration_stats)[3:ncol(physician_integration_stats)], sep = "")
  )


  most_common_primary_care <- left_join(
    data_selected,
    physician_integration_stats,
    by = c(
      "most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year"
    )
  ) %>% as.data.table()

  most_common_primary_care <- most_common_primary_care[, -c("most_common_physician_PRF_PHYSN_NPI")]

  rename_last(
    most_common_primary_care,
    ncol(physician_integration_stats) - 2,
    paste("most_common_primary_care_physician_", colnames(physician_integration_stats)[3:ncol(physician_integration_stats)], sep = "")
  )

  physician_data <-
    full_join(most_common_physician[, -("most_common_physician_PRF_PHYSN_NPI")],
      most_common_primary_care[, -("most_common_primary_care_physician_PRF_PHYSN_NPI")],
      by = c("DESY_SORT_KEY", "year")
    )
  result <- full_join(data,
    physician_data,
    by = c("DESY_SORT_KEY", "year")
  ) %>%
    as.data.table()


  return(result)
}

summary_with_physician_integration_stats <- add_integration_status(
  data = summary_with_npi,
  physician_integration_stats = physician_integration_stats
)

In [ ]:
head(summary_with_physician_integration_stats)

In [ ]:
write.fst(summary_with_physician_integration_stats, "summary_with_physician_integration_stats.fst")

In [149]:
summary_with_npi[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,56,11,1578594818,56,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11
100005325,2014,13,2,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,14,20,140,2,1,77,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V,20140125,20140125,V,2014,1578594818,4,11,1578594818,4,11


# Comparisons and analyses

## Reading the patient and physician integration results

In [5]:
yearly_calculations <-
  read_fst("calculation_results.fst", as.data.table = T)

yearly_calculations <- unique(yearly_calculations)

In [ ]:
yearly_calculations[DESY_SORT_KEY == "100005325"]

## Adding metropolitan status
I will add the metropolitan statuses of the patient counties using the USDA data

In [11]:
rural_urban_data <- readxl::read_xls("physician_data/ruralurbancodes2013.xls") %>% as.data.table()
head(rural_urban_data)
cross_walk_rural_urban <- read.csv(file = "physician_data/xwalk2018.csv") %>% as.data.table()
head(cross_walk_rural_urban)

FIPS,State,County_Name,Population_2010,RUCC_2013,Description
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
01001,AL,Autauga County,54571,2,"Metro - Counties in metro areas of 250,000 to 1 million population"
01003,AL,Baldwin County,182265,3,"Metro - Counties in metro areas of fewer than 250,000 population"
01005,AL,Barbour County,27457,6,"Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area"
01007,AL,Bibb County,22915,1,Metro - Counties in metro areas of 1 million population or more
01009,AL,Blount County,57322,1,Metro - Counties in metro areas of 1 million population or more
01011,AL,Bullock County,10914,6,"Nonmetro - Urban population of 2,500 to 19,999, adjacent to a metro area"


County.Name,State,SSACD,FIPS.County.Code,CBSA,CBSA.Name
<chr>,<chr>,<int>,<int>,<int>,<chr>
AUTAUGA,AL,1000,1001,33860,"Montgomery, AL"
BALDWIN,AL,1010,1003,19300,"Daphne-Fairhope-Foley, AL"
BARBOUR,AL,1020,1005,NA,
BIBB,AL,1030,1007,13820,"Birmingham-Hoover, AL"
BLOUNT,AL,1040,1009,13820,"Birmingham-Hoover, AL"
BULLOCK,AL,1050,1011,NA,


In [12]:
yearly_calculations[, SSACD := as.integer(paste(STATE_CODE, COUNTY_CODE, sep = ""))]
yearly_calculations <- left_join(yearly_calculations, cross_walk_rural_urban, by = "SSACD") %>%
  mutate(FIPS.County.Code = as.character(FIPS.County.Code)) %>%
  left_join(., rural_urban_data, by = c("FIPS.County.Code" = "FIPS")) %>%
  as.data.table()
yearly_calculations[, c("SSACD", "County.Name", "State.x", "FIPS.County.Code", "CBSA", "CBSA.Name", "State.y", "County_Name", "Population_2010", "Description") := NULL]
head(yearly_calculations)


# STATE_CODE	COUNTY_CODE	SEX_CODE	RACE_CODE

Warning message in eval(jsub, SDenv, parent.frame()):
“NAs introduced by coercion”


DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1
100000015,2014,10,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,14,22,160,1,1,77,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,16,16,22,22,27,27,1.00000000,1.0000000,1.00000000,1
100000015,2015,6,2,TRUE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,15,22,160,1,1,78,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1770514119,4,11,1770514119,4,11,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,48,48,57,57,57,57,1.00000000,1.0000000,1.00000000,1
100000015,2016,9,2,TRUE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2016,22,160,1,1,79,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1730170630,4,06,1730170630,4,11,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,0,21,0,21,4,41,0.00000000,0.0000000,0.09756098,1
100000015,2017,14,3,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2017,22,160,1,1,80,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1043207350,3,11,1043207350,3,11,58,58,68,68,69,69,1.00000000,1.0000000,1.00000000,58,58,68,68,69,69,1.00000000,1.0000000,1.00000000,1
100000015,2018,16,2,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,TRUE,2018,22,160,1,1,81,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,10432

## Adding sex, race, and state names from codes

In [13]:
add_personal_details <- function(data) {
  require(tidyverse)
  require(dtplyr)
  require(lubridate)

  result <- data %>%
    # left_join(.,census_and_state_codes[,-1],by=c("STATE_CODE"="state_code"))%>%
    left_join(., race_codes, by = c("RACE_CODE" = "race_code")) %>%
    left_join(., sex_codes, by = c("SEX_CODE" = "sex_code")) %>%
    mutate(
      age_group = case_when(
        AGE < 75 & AGE > 64 ~ "65-74",
        AGE > 74 & AGE < 85 ~ "75-84",
        AGE > 84 ~ "85+"
      ),
      urban = (RUCC_2013 <= 3)
    ) %>%
    as.data.table()

  return(result)
}

# STATE_CODE	COUNTY_CODE	SEX_CODE	RACE_CODE

In [14]:
yearly_calculations <- add_personal_details(yearly_calculations)

Loading required package: lubridate


Attaching package: ‘lubridate’


The following object is masked from ‘package:reshape’:

    stamp


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week, yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




## Adding HMO indicator and buy in sums

In [15]:
yearly_calculations[, HMO_INDICATOR_sum := sum(
  HMO_INDICATOR01 == 0,
  HMO_INDICATOR02 == 0,
  HMO_INDICATOR03 == 0,
  HMO_INDICATOR04 == 0,
  HMO_INDICATOR05 == 0,
  HMO_INDICATOR06 == 0,
  HMO_INDICATOR07 == 0,
  HMO_INDICATOR08 == 0,
  HMO_INDICATOR09 == 0,
  HMO_INDICATOR10 == 0,
  HMO_INDICATOR11 == 0,
  HMO_INDICATOR12 == 0,
  na.rm = T
), by = 1:nrow(yearly_calculations)]

yearly_calculations[, ENTITLEMENT_BUY_IN_IND_sum := sum(
  ENTITLEMENT_BUY_IN_IND01 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND02 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND03 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND04 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND05 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND06 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND07 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND08 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND09 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND10 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND11 %in% c("3", "C"),
  ENTITLEMENT_BUY_IN_IND12 %in% c("3", "C"),
  na.rm = T
), by = 1:nrow(yearly_calculations)]

## Adding months alive within the year of death

In [16]:
yearly_calculations[, months_alive := as.numeric(substr(DATE_OF_DEATH, 5, 6))]

## Filtering the data for analysis

In [17]:
length(yearly_calculations$DESY_SORT_KEY)

[1] 14342911

In [18]:
data_for_modelling_filter <- function(data) {
  library(tidyverse)
  library(dtplyr)

  data %>%
    filter(STATE_CODE %!in% non_us_state_codes &

      AGE >= 65 &

      ((!is.na(months_alive) & HMO_INDICATOR_sum >= months_alive) |
        (is.na(months_alive) & HMO_INDICATOR_sum == 12)) &

      ((!is.na(months_alive) & ENTITLEMENT_BUY_IN_IND_sum >= months_alive) |
        (is.na(months_alive) & ENTITLEMENT_BUY_IN_IND_sum == 12))) %>%
    as.data.table()
}

In [19]:
yearly_calculations <- data_for_modelling_filter(yearly_calculations)

In [20]:
length(yearly_calculations$DESY_SORT_KEY)

[1] 10012783

# Add MDPPAS

## Read from SAS file and convert to fst

In [ ]:
library(haven)
mdppas_data <- read_sas(
  data_file =
    "/work/postresearch/Shared/Data_raw/Medicare/MDPPAS/mdppas0819.sas7bdat"
)

In [ ]:
head(mdppas_data)

In [4]:
write_fst(x = mdppas_data, path = "/work/postresearch/Shared/Data_raw/Medicare/MDPPAS/mdppas.fst", compress = 0)

## Read from fst

In [3]:
mdppas_data <-
  read_fst(path = "/work/postresearch/Shared/Data_raw/Medicare/MDPPAS/mdppas.fst", as.data.table = T)

In [4]:
head(mdppas_data)

npi,year,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1003000126,2009,NA,0,NU00286134,UPMC WESTERN MARYLAND CORPORATION,,,ENKESHAFI,ARDALAN,,Hospitalist,C6,5,M,11,Internal Medicine,1,2,0.0000000000,1.0000000,0.000000000,0.0000000000,0,0,0,0,0,0.000000000,MD,0,1,19060,"Cumberland, MD-WV",0,405,28669.86,176,000000001111,405,28669.86,176,,NA,NA,NA,1968,0,0
1003000126,2010,1,0,NU00286134,UPMC WESTERN MARYLAND CORPORATION,,,ENKESHAFI,ARDALAN,,Hospitalist,C6,5,M,11,Internal Medicine,1,2,0.0000000000,0.9922280,0.003454231,0.0000000000,0,0,0,0,0,0.004317789,MD,0,1,19060,"Cumberland, MD-WV",0,1158,101834.65,510,111111111111,1158,101834.65,510,,NA,NA,NA,1968,0,4
1003000126,2011,1,0,NU00286134,UPMC WESTERN MARYLAND CORPORATION,,,ENKESHAFI,ARDALAN,,Hospitalist,C6,5,M,11,Internal Medicine,1,2,0.0000000000,0.9738520,0.022321429,0.0006377551,0,0,0,0,0,0.003188776,MD,0,1,19060,"Cumberland, MD-WV",0,1568,130586.46,685,111111111111,1568,130586.46,685,,NA,NA,NA,1968,0,35
1003000126,2012,1,0,NU00286134,UPMC WESTERN MARYLAND CORPORATION,,,ENKESHAFI,ARDALAN,,Hospitalist,C6,5,M,11,Internal Medicine,1,2,0.0008045052,0.9903459,0.005631537,0.0000000000,0,0,0,0,0,0.003218021,MD,0,1,19060,"Cumberland, MD-WV",0,1243,113956.76,581,111111111111,1243,113956.76,581,,NA,NA,NA,1968,1,7
1003000126,2013,1,1,NU00145029,HOSPITALIST MEDICINE PHYSICIANS OF MARYLAND PC,NU00286134,UPMC WESTERN MARYLAND CORPORATION,ENKESHAFI,ARDALAN,,Hospitalist,C6,5,M,11,Internal Medicine,1,2,0.0000000000,0.9836066,0.006678810,0.0042501518,0,0,0,0,0,0.005464481,MD,0,1,12580,"Baltimore-Columbia-Towson, MD",1,1647,146521.84,660,000111111111,1217,107567.04,487,111101000000,430,38954.8,197,1968,0,11
1003000126,2014,1,1,NU00145029,HOSPITALIST MEDICINE PHYSICIANS OF MARYLAND PC,,,ENKESHAFI,ARDALAN,,Hospitalist,C6,5,M,11,Internal Medicine,1,2,0.0000000000,0.9930884,0.001818843,0.0000000000,0,0,0,0,0,0.005092761,MD,0,1,12580,"Baltimore-Columbia-Towson, MD",0,2749,248222.87,907,111111111111,2749,248222.87,907,,NA,NA,NA,1968,0,5


## Add to calculations and integration stats

In [7]:
yearly_calculations <- read_fst("yearly_calculations_cleaned.fst", as.data.table = T)

In [9]:
yearly_calculations <-
  left_join(yearly_calculations, mdppas_data,
    by = c("most_common_primary_care_physician_PRF_PHYSN_NPI" = "npi", "year" = "year")
  ) %>% as.data.table()
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
100000015,2013,7,1,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,FALSE,13,22,160,1,1,76,0,0,3,3,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,1285600932,2,11,1285600932,2,11,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,7,96,62,242,66,341,0.07291667,0.2561983,0.19354839,1,White,Male,75-84,TRUE,12,12,NA,1,0,0,NU00015773,"SALIL K MIDHA, MD, PC",,,MIDHA,SALIL,K,Internal Medicine,11,1,M,,NONE,1,NA,0.6718430,0.19323176,0.13492529,0.000000000,0,0,0,0,0,0.00000000,MA,0,1,14460,"Boston-Cambridge-Newton, MA-NH",0,6826,510422.97,1608,111111

In [10]:
write_fst(x = yearly_calculations, "yearly_calculations_cleaned_with_mdppas.fst", compress = 0)

In [5]:
physician_integration_stats <- read.fst("./physician_integration_stats_before_join.fst", as.data.table = T)

In [6]:
physician_integration_stats <-
  left_join(physician_integration_stats, mdppas_data,
    by = c("PRF_PHYSN_NPI" = "npi", "year" = "year")
  ) %>% as.data.table()

In [7]:
physician_integration_stats[PRF_PHYSN_NPI == 1770514119]

PRF_PHYSN_NPI,year,in_facility_visits_count,in_all_visits_count,in_facility_non_exclusive_HCPCS_count,in_all_non_exclusive_HCPCS_count,in_facility_count,in_all_count,in_facility_visits_prp,in_facility_non_exclusive_HCPCS_prp,in_facility_prp,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd
<chr>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1770514119,2013,1,1,4,4,18,18,1,1,1,1,0,NU00014364,NORTH SHORE PHYSICIANS GROUP INC,NU00111340,"SOUTHCOAST PHYSICIANS GROUP, INC..",MOPALA,PRASHANTH,R,Internal Medicine,11,1,M,,NONE,1,NA,0,0.79761905,0.1794218,0.022959184,0,0,0,0,0,0.00000000,MA,0,1,14460,"Boston-Cambridge-Newton, MA-NH",1,1176,99079.09,476,111111110001,1115,94208.49,431,000001111000,44,3904.68,30,1976,0,211
1770514119,2014,16,16,22,22,27,27,1,1,1,1,1,NU00013485,THE GENERAL HOSPITAL CORPORATION,NU00014364,NORTH SHORE PHYSICIANS GROUP INC,MOPALA,PRASHANTH,R,Internal Medicine,11,1,M,,NONE,1,NA,0,0.08988764,0.8157303,0.065168539,0,0,0,0,0,0.02921348,MA,0,1,14460,"Boston-Cambridge-Newton, MA-NH",0,445,45397.52,241,111111111111,313,28835.45,140,111111111111,100,14872.70,96,1976,0,363
1770514119,2015,48,48,57,57,57,57,1,1,1,1,1,NU00013485,THE GENERAL HOSPITAL CORPORATION,NU00013835,MASSACHUSETTS GENERAL PHYSICIANS ORGANIZATION INC,MOPALA,PRASHANTH,R,Internal Medicine,11,1,M,,NONE,1,NA,0,0.11174242,0.8863636,0.001893939,0,0,0,0,0,0.00000000,MA,0,1,14460,"Boston-Cambridge-Newton, MA-NH",0,528,47201.44,207,111111111111,439,39579.08,182,100111111111,76,5296.59,29,1976,0,468
1770514119,2016,17,17,20,20,20,20,1,1,1,1,1,NU00013485,THE GENERAL HOSPITAL CORPORATION,NU00110881,DARTMOUTH HITCHCOCK CLINIC,MOPALA,PRASHANTH,R,Internal Medicine,11,1,M,,NONE,1,NA,0,0.29431438,0.7056856,0.000000000,0,0,0,0,0,0.00000000,MA,1,1,14460,"Boston-Cambridge-Newton, MA-NH",1,299,31555.09,201,111110000000,189,16219.13,104,000000111111,96,15114.46,96,1976,0,211
1770514119,2017,0,0,0,0,3,3,NaN,NaN,1,1,0,NU00110881,DARTMOUTH HITCHCOCK CLINIC,,,MOPALA,PRASHANTH,R,Internal Medicine,11,1,M,,NONE,1,NA,0,0.74175824,0.2582418,0.000000000,0,0,0,0,0,0.00000000,NH,0,2,17200,"Claremont-Lebanon, NH-VT",0,182,27219.00,177,111111111111,182,27219.00,177,,NA,NA,NA,1976,0,47


# Finding changes in physician patient pools

In [8]:
physician_integration_change_finder <- function(data, integration_threshold = 0.75, independece_threshold = 0.75) {
  require(tidyverse)
  require(dtplyr)

  result <- data %>%
    mutate(
      integrated = (in_facility_non_exclusive_HCPCS_prp >= integration_threshold | vikeyword == 1)
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      integration_change =
        case_when(
          integrated == T & lag(integrated) == F ~ "newly_integrated",
          integrated == F & lag(integrated) == T ~ "newly_disintegrated",
          is.na(integrated) & lag(integrated) == F ~ "remained_independent",
          integrated == F & lag(integrated) == F ~ "remained_independent",
          integrated == T & lag(integrated) == T ~ "remained_integrated",
          integrated == T & is.na(lag(integrated)) & lag(integrated, 2) == T ~ "remained_integrated",
          integrated == T & is.na(lag(integrated)) & lag(integrated, 2) == F ~ "newly_integrated",
          is.na(integrated) & lag(integrated) == T ~ "remained_integrated",
          year == 2013 & integrated == T ~ "remained_integrated",
          year == 2013 & integrated == F ~ "remained_independent"
        ),
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      sum_integration_changes = sum(integration_change %in% c("newly_integrated", "newly_disintegrated"), na.rm = T),
      integrated_once_and_stayed =
        sum(integration_change == "newly_disintegrated", na.rm = T) == 0 &
          sum(integration_change == "newly_integrated", na.rm = T) == 1,
      never_integrated =
        sum(integration_change == "newly_integrated", na.rm = T) == 0 &
          sum(integration_change == "remained_integrated", na.rm = T) == 0 &
          sum(integration_change == "newly_disintegrated", na.rm = T) == 0
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      year_integrated = case_when(integration_change == "newly_integrated" & integrated_once_and_stayed == T ~ year)
    ) %>%
    group_by(PRF_PHYSN_NPI) %>%
    mutate(
      year_integrated = year_integrated[which(!is.na(year_integrated))]
    ) %>%
    mutate(years_from_integration = year - year_integrated) %>%
    as.data.table()
}

In [9]:
physician_integration_stats_changes <- physician_integration_change_finder(physician_integration_stats)

In [26]:
yearly_calculations <- read.fst("./yearly_calculations_cleaned_with_mdppas.fst", as.data.table = T)

In [27]:
yearly_calculations <- left_join(
  yearly_calculations,
  physician_integration_stats_changes[, .(PRF_PHYSN_NPI, year, integrated, integration_change, sum_integration_changes, integrated_once_and_stayed, never_integrated, year_integrated, years_from_integration)],
  by = c("most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year")
) %>% as.data.table()

In [28]:
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}

rename_last(yearly_calculations, 7, paste("most_common_primary_care_physician_", colnames(yearly_calculations[, (ncol(yearly_calculations) - 6):ncol(yearly_calculations)]), sep = ""))

In [64]:
yearly_calculations <- yearly_calculations %>%
  group_by(DESY_SORT_KEY) %>%
  mutate(
    most_common_primary_care_physician_changed = case_when(
      most_common_primary_care_physician_PRF_PHYSN_NPI != lag(most_common_primary_care_physician_PRF_PHYSN_NPI) ~ T,
      most_common_primary_care_physician_PRF_PHYSN_NPI == lag(most_common_primary_care_physician_PRF_PHYSN_NPI) ~ F,
      year == 2013 ~ F
    )
  ) %>%
  as.data.table()

In [65]:
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration,exit_enter_status
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<i

## Adding last year's physician's integration status

In [31]:
yearly_calculations <- yearly_calculations %>%
  group_by(DESY_SORT_KEY) %>%
  arrange(year) %>%
  mutate(
    last_year_most_common_primary_care_physician_PRF_PHYSN_NPI =
      lag(most_common_primary_care_physician_PRF_PHYSN_NPI, 1)
  ) %>%
  as.data.table()

yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<int>,<lgl>,<lgl>,<dbl>,<dbl>,<lgl>,<chr>
1000

In [32]:
yearly_calculations <- left_join(
  yearly_calculations,
  physician_integration_stats_changes[, .(PRF_PHYSN_NPI, year, integrated, integration_change, sum_integration_changes, integrated_once_and_stayed, never_integrated, year_integrated, years_from_integration)],
  by = c("last_year_most_common_primary_care_physician_PRF_PHYSN_NPI" = "PRF_PHYSN_NPI", "year" = "year")
) %>% as.data.table()

In [33]:
rename_last <- function(data, how_many, new_names) {
  total_cols <- ncol(data)
  setnames(data, (total_cols - how_many + 1):(total_cols), new_names)
}

rename_last(yearly_calculations, 7, paste("last_year_most_common_primary_care_physician_", colnames(yearly_calculations[, (ncol(yearly_calculations) - 6):ncol(yearly_calculations)]), sep = ""))

In [34]:
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<d

In [36]:
write.fst(yearly_calculations, "yearly_calculations_with_integration_changes_mdppas.fst")

# Adding claim level file

## Reading the CSVs

In [ ]:
claim_carrier_2013 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsj_lds_5_2013.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2014 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsj_lds_5_2014.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2015 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsj_lds_5_2015.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2016 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsk_lds_5_2016.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2017 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsk_lds_5_2017.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2018 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsk_lds_5_2018.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2019 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsk_lds_5_2019.csv", num_threads = numcores) %>% as.data.table()
claim_carrier_2020 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/car_claimsk_lds_5_2020.csv", num_threads = numcores) %>% as.data.table()

claim_inpatient_2013 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsj_lds_5_2013.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2014 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsj_lds_5_2014.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2015 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsj_lds_5_2015.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2016 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsk_lds_5_2016.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2017 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsk_lds_5_2017.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2018 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsk_lds_5_2018.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2019 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsk_lds_5_2019.csv", num_threads = numcores) %>% as.data.table()
claim_inpatient_2020 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/inp_claimsk_lds_5_2020.csv", num_threads = numcores) %>% as.data.table()

claim_outpatient_2013 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsj_lds_5_2013.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2014 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsj_lds_5_2014.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2015 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsj_lds_5_2015.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2016 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsk_lds_5_2016.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2017 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsk_lds_5_2017.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2018 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsk_lds_5_2018.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2019 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsk_lds_5_2019.csv", num_threads = numcores) %>% as.data.table()
claim_outpatient_2020 <- vroom(file = "/work/postresearch/Shared/Data_raw/Medicare/Claims/CSV/out_claimsk_lds_5_2020.csv", num_threads = numcores) %>% as.data.table()

New names:
• `1` -> `1...2`
• `1` -> `1...8`
• `3.35` -> `3.35...9`
• `0.00` -> `0.00...10`
• `3.35` -> `3.35...14`
• `0.00` -> `0.00...15`
• `496` -> `496...21`
• `9` -> `9...22`
• `496` -> `496...23`
• `9` -> `9...24`
• `9` -> `9...26`
• `9` -> `9...28`
• `` -> `...29`
• `` -> `...30`
• `` -> `...31`
• `` -> `...32`
• `` -> `...33`
• `` -> `...34`
• `` -> `...35`
• `` -> `...36`
• `` -> `...37`
• `` -> `...38`
• `` -> `...39`
• `` -> `...40`
• `` -> `...41`
• `` -> `...42`
• `` -> `...43`
• `` -> `...44`
• `` -> `...45`
• `` -> `...46`
• `1` -> `1...48`
• `1` -> `1...49`
Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 44560480 Columns: 52
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): O, 01, 14202, 1...8, G31613, A, A2151101, 496...21, 496...23, 7862, 51889, ...29, 160, 22
dbl (30): 100000015, 1...2, 20130425, 71, 3.35...9, 0.00...10, 1659344091, 3.35...14, 0.00...15, 419.00, 112.52, 108.25, 

In [ ]:
colnames_claim_carrier_2011_2015 <-
  c("DESY_SORT_KEY", "CLAIM_NO", "CLM_THRU_DT", "NCH_NEAR_LINE_REC_IDENT_CD", "NCH_CLM_TYPE_CD", "CLM_DISP_CD", "CARR_NUM", "CARR_CLM_PMT_DNL_CD", "CLM_PMT_AMT", "CARR_CLM_PRMRY_PYR_PD_AMT", "RFR_PHYSN_UPIN", "RFR_PHYSN_NPI", "CARR_CLM_PRVDR_ASGNMT_IND_SW", "PROV_PMT", "BENE_PMT", "SBMTCHRG", "ALOWCHRG", "DEDAPPLY", "HCPCS_YR", "CARR_CLM_RFRNG_PIN_NUM", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12", "DOB_DT", "GNDR_CD", "BENE_RACE_CD", "BENE_CNTY_CD", "BENE_STATE_CD", "CWF_BENE_MDCR_STUS_CD")

colnames_claim_carrier_2016_2020 <-
  c("DESY_SORT_KEY", "CLAIM_NO", "CLM_THRU_DT", "NCH_NEAR_LINE_REC_IDENT_CD", "NCH_CLM_TYPE_CD", "CLM_DISP_CD", "CARR_NUM", "CARR_CLM_PMT_DNL_CD", "CLM_PMT_AMT", "CARR_CLM_PRMRY_PYR_PD_AMT", "RFR_PHYSN_UPIN", "RFR_PHYSN_NPI", "CARR_CLM_PRVDR_ASGNMT_IND_SW", "NCH_CLM_PRVDR_PMT_AMT", "NCH_CLM_BENE_PMT_AMT", "NCH_CARR_CLM_SBMTD_CHRG_AMT", "NCH_CARR_CLM_ALOWD_AMT", "CARR_CLM_CASH_DDCTBL_APLD_AMT", "CARR_CLM_HCPCS_YR_CD", "CARR_CLM_RFRNG_PIN_NUM", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12", "DOB_DT", "GNDR_CD", "BENE_RACE_CD", "BENE_CNTY_CD", "BENE_STATE_CD", "CWF_BENE_MDCR_STUS_CD", "CLM_BENE_PD_AMT", "CPO_PRVDR_NUM", "CPO_ORG_NPI_NUM", "CARR_CLM_BLG_NPI_NUM", "ACO_ID_NUM")

colnames_claim_inpatient_2011_2015 <-
  c("DESY_SORT_KEY", "CLAIM_NO", "PRVDR_NUM", "CLM_THRU_DT", "NCH_NEAR_LINE_REC_IDENT_CD", "NCH_CLM_TYPE_CD", "CLAIM_QUERY_CODE", "CLM_FAC_TYPE_CD", "CLM_SRVC_CLSFCTN_TYPE_CD", "CLM_FREQ_CD", "FI_NUM", "CLM_MDCR_NON_PMT_RSN_CD", "CLM_PMT_AMT", "NCH_PRMRY_PYR_CLM_PD_AMT", "NCH_PRMRY_PYR_CD", "FI_CLM_ACTN_CD", "PRVDR_STATE_CD", "ORG_NPI_NUM", "AT_PHYSN_UPIN", "AT_PHYSN_NPI", "OP_PHYSN_UPIN", "OP_PHYSN_NPI", "OT_PHYSN_UPIN", "OT_PHYSN_NPI", "CLM_MCO_PD_SW", "PTNT_DSCHRG_STUS_CD", "CLM_PPS_IND_CD", "CLM_TOT_CHRG_AMT", "CLM_ADMSN_DT", "CLM_IP_ADMSN_TYPE_CD", "CLM_SRC_IP_ADMSN_CD", "NCH_PTNT_STATUS_IND_CD", "CLM_PASS_THRU_PER_DIEM_AMT", "NCH_BENE_IP_DDCTBL_AMT", "NCH_BENE_PTA_COINSRNC_LBLTY_AM", "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM", "NCH_PROFNL_CMPNT_CHRG_AMT", "NCH_IP_NCVRD_CHRG_AMT", "CLM_TOT_PPS_CPTL_AMT", "CLM_PPS_CPTL_FSP_AMT", "CLM_PPS_CPTL_OUTLIER_AMT", "CLM_PPS_CPTL_DSPRPRTNT_SHR_AMT", "CLM_PPS_CPTL_IME_AMT", "CLM_PPS_CPTL_EXCPTN_AMT", "CLM_PPS_OLD_CPTL_HLD_HRMLS_AMT", "CLM_PPS_CPTL_DRG_WT_NUM", "CLM_UTLZTN_DAY_CNT", "BENE_TOT_COINSRNC_DAYS_CNT", "BENE_LRD_USED_CNT", "CLM_NON_UTLZTN_DAYS_CNT", "NCH_BLOOD_PNTS_FRNSHD_QTY", "NCH_VRFD_NCVRD_STAY_FROM_DT", "NCH_VRFD_NCVRD_STAY_THRU_DT", "NCH_BENE_MDCR_BNFTS_EXHTD_DT_I", "NCH_BENE_DSCHRG_DT", "CLM_DRG_CD", "CLM_DRG_OUTLIER_STAY_CD", "NCH_DRG_OUTLIER_APRVD_PMT_AMT", "ADMTG_DGNS_CD", "ADMTG_DGNS_VRSN_CD", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "CLM_POA_IND_SW1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "CLM_POA_IND_SW2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "CLM_POA_IND_SW3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "CLM_POA_IND_SW4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "CLM_POA_IND_SW5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "CLM_POA_IND_SW6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "CLM_POA_IND_SW7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "CLM_POA_IND_SW8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "CLM_POA_IND_SW9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "CLM_POA_IND_SW10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "CLM_POA_IND_SW11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12", "CLM_POA_IND_SW12", "ICD_DGNS_CD13", "ICD_DGNS_VRSN_CD13", "CLM_POA_IND_SW13", "ICD_DGNS_CD14", "ICD_DGNS_VRSN_CD14", "CLM_POA_IND_SW14", "ICD_DGNS_CD15", "ICD_DGNS_VRSN_CD15", "CLM_POA_IND_SW15", "ICD_DGNS_CD16", "ICD_DGNS_VRSN_CD16", "CLM_POA_IND_SW16", "ICD_DGNS_CD17", "ICD_DGNS_VRSN_CD17", "CLM_POA_IND_SW17", "ICD_DGNS_CD18", "ICD_DGNS_VRSN_CD18", "CLM_POA_IND_SW18", "ICD_DGNS_CD19", "ICD_DGNS_VRSN_CD19", "CLM_POA_IND_SW19", "ICD_DGNS_CD20", "ICD_DGNS_VRSN_CD20", "CLM_POA_IND_SW20", "ICD_DGNS_CD21", "ICD_DGNS_VRSN_CD21", "CLM_POA_IND_SW21", "ICD_DGNS_CD22", "ICD_DGNS_VRSN_CD22", "CLM_POA_IND_SW22", "ICD_DGNS_CD23", "ICD_DGNS_VRSN_CD23", "CLM_POA_IND_SW23", "ICD_DGNS_CD24", "ICD_DGNS_VRSN_CD24", "CLM_POA_IND_SW24", "ICD_DGNS_CD25", "ICD_DGNS_VRSN_CD25", "CLM_POA_IND_SW25", "FST_DGNS_E_CD", "FST_DGNS_E_VRSN_CD", "ICD_DGNS_E_CD1", "ICD_DGNS_E_VRSN_CD1", "CLM_E_POA_IND_SW1", "ICD_DGNS_E_CD2", "ICD_DGNS_E_VRSN_CD2", "CLM_E_POA_IND_SW2", "ICD_DGNS_E_CD3", "ICD_DGNS_E_VRSN_CD3", "CLM_E_POA_IND_SW3", "ICD_DGNS_E_CD4", "ICD_DGNS_E_VRSN_CD4", "CLM_E_POA_IND_SW4", "ICD_DGNS_E_CD5", "ICD_DGNS_E_VRSN_CD5", "CLM_E_POA_IND_SW5", "ICD_DGNS_E_CD6", "ICD_DGNS_E_VRSN_CD6", "CLM_E_POA_IND_SW6", "ICD_DGNS_E_CD7", "ICD_DGNS_E_VRSN_CD7", "CLM_E_POA_IND_SW7", "ICD_DGNS_E_CD8", "ICD_DGNS_E_VRSN_CD8", "CLM_E_POA_IND_SW8", "ICD_DGNS_E_CD9", "ICD_DGNS_E_VRSN_CD9", "CLM_E_POA_IND_SW9", "ICD_DGNS_E_CD10", "ICD_DGNS_E_VRSN_CD10", "CLM_E_POA_IND_SW10", "ICD_DGNS_E_CD11", "ICD_DGNS_E_VRSN_CD11", "CLM_E_POA_IND_SW11", "ICD_DGNS_E_CD12", "ICD_DGNS_E_VRSN_CD12", "CLM_E_POA_IND_SW12", "ICD_PRCDR_CD1", "ICD_PRCDR_VRSN_CD1", "PRCDR_DT1", "ICD_PRCDR_CD2", "ICD_PRCDR_VRSN_CD2", "PRCDR_DT2", "ICD_PRCDR_CD3", "ICD_PRCDR_VRSN_CD3", "PRCDR_DT3", "ICD_PRCDR_CD4", "ICD_PRCDR_VRSN_CD4", "PRCDR_DT4", "ICD_PRCDR_CD5", "ICD_PRCDR_VRSN_CD5", "PRCDR_DT5", "ICD_PRCDR_CD6", "ICD_PRCDR_VRSN_CD6", "PRCDR_DT6", "ICD_PRCDR_CD7", "ICD_PRCDR_VRSN_CD7", "PRCDR_DT7", "ICD_PRCDR_CD8", "ICD_PRCDR_VRSN_CD8", "PRCDR_DT8", "ICD_PRCDR_CD9", "ICD_PRCDR_VRSN_CD9", "PRCDR_DT9", "ICD_PRCDR_CD10", "ICD_PRCDR_VRSN_CD10", "PRCDR_DT10", "ICD_PRCDR_CD11", "ICD_PRCDR_VRSN_CD11", "PRCDR_DT11", "ICD_PRCDR_CD12", "ICD_PRCDR_VRSN_CD12", "PRCDR_DT12", "ICD_PRCDR_CD13", "ICD_PRCDR_VRSN_CD13", "PRCDR_DT13", "ICD_PRCDR_CD14", "ICD_PRCDR_VRSN_CD14", "PRCDR_DT14", "ICD_PRCDR_CD15", "ICD_PRCDR_VRSN_CD15", "PRCDR_DT15", "ICD_PRCDR_CD16", "ICD_PRCDR_VRSN_CD16", "PRCDR_DT16", "ICD_PRCDR_CD17", "ICD_PRCDR_VRSN_CD17", "PRCDR_DT17", "ICD_PRCDR_CD18", "ICD_PRCDR_VRSN_CD18", "PRCDR_DT18", "ICD_PRCDR_CD19", "ICD_PRCDR_VRSN_CD19", "PRCDR_DT19", "ICD_PRCDR_CD20", "ICD_PRCDR_VRSN_CD20", "PRCDR_DT20", "ICD_PRCDR_CD21", "ICD_PRCDR_VRSN_CD21", "PRCDR_DT21", "ICD_PRCDR_CD22", "ICD_PRCDR_VRSN_CD22", "PRCDR_DT22", "ICD_PRCDR_CD23", "ICD_PRCDR_VRSN_CD23", "PRCDR_DT23", "ICD_PRCDR_CD24", "ICD_PRCDR_VRSN_CD24", "PRCDR_DT24", "ICD_PRCDR_CD25", "ICD_PRCDR_VRSN_CD25", "PRCDR_DT25", "DOB_DT", "GNDR_CD", "BENE_RACE_CD", "BENE_CNTY_CD", "BENE_STATE_CD", "CWF_BENE_MDCR_STUS_CD")

colnames_claim_inpatient_2016_2020 <-
  c("DESY_SORT_KEY", "CLAIM_NO", "PRVDR_NUM", "CLM_THRU_DT", "NCH_NEAR_LINE_REC_IDENT_CD", "NCH_CLM_TYPE_CD", "CLAIM_QUERY_CODE", "CLM_FAC_TYPE_CD", "CLM_SRVC_CLSFCTN_TYPE_CD", "CLM_FREQ_CD", "FI_NUM", "CLM_MDCR_NON_PMT_RSN_CD", "CLM_PMT_AMT", "NCH_PRMRY_PYR_CLM_PD_AMT", "NCH_PRMRY_PYR_CD", "FI_CLM_ACTN_CD", "PRVDR_STATE_CD", "ORG_NPI_NUM", "AT_PHYSN_UPIN", "AT_PHYSN_NPI", "AT_PHYSN_SPCLTY_CD", "OP_PHYSN_UPIN", "OP_PHYSN_NPI", "OP_PHYSN_SPCLTY_CD", "OT_PHYSN_UPIN", "OT_PHYSN_NPI", "OT_PHYSN_SPCLTY_CD", "RNDRNG_PHYSN_NPI", "RNDRNG_PHYSN_SPCLTY_CD", "CLM_MCO_PD_SW", "PTNT_DSCHRG_STUS_CD", "CLM_PPS_IND_CD", "CLM_TOT_CHRG_AMT", "CLM_ADMSN_DT", "CLM_IP_ADMSN_TYPE_CD", "CLM_SRC_IP_ADMSN_CD", "NCH_PTNT_STATUS_IND_CD", "CLM_PASS_THRU_PER_DIEM_AMT", "NCH_BENE_IP_DDCTBL_AMT", "NCH_BENE_PTA_COINSRNC_LBLTY_AM", "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM", "NCH_PROFNL_CMPNT_CHRG_AMT", "NCH_IP_NCVRD_CHRG_AMT", "CLM_TOT_PPS_CPTL_AMT", "CLM_PPS_CPTL_FSP_AMT", "CLM_PPS_CPTL_OUTLIER_AMT", "CLM_PPS_CPTL_DSPRPRTNT_SHR_AMT", "CLM_PPS_CPTL_IME_AMT", "CLM_PPS_CPTL_EXCPTN_AMT", "CLM_PPS_OLD_CPTL_HLD_HRMLS_AMT", "CLM_PPS_CPTL_DRG_WT_NUM", "CLM_UTLZTN_DAY_CNT", "BENE_TOT_COINSRNC_DAYS_CNT", "BENE_LRD_USED_CNT", "CLM_NON_UTLZTN_DAYS_CNT", "NCH_BLOOD_PNTS_FRNSHD_QTY", "NCH_VRFD_NCVRD_STAY_FROM_DT", "NCH_VRFD_NCVRD_STAY_THRU_DT", "NCH_BENE_MDCR_BNFTS_EXHTD_DT_I", "NCH_BENE_DSCHRG_DT", "CLM_DRG_CD", "CLM_DRG_OUTLIER_STAY_CD", "NCH_DRG_OUTLIER_APRVD_PMT_AMT", "ADMTG_DGNS_CD", "PRNCPAL_DGNS_CD", "ICD_DGNS_CD1", "CLM_POA_IND_SW1", "ICD_DGNS_CD2", "CLM_POA_IND_SW2", "ICD_DGNS_CD3", "CLM_POA_IND_SW3", "ICD_DGNS_CD4", "CLM_POA_IND_SW4", "ICD_DGNS_CD5", "CLM_POA_IND_SW5", "ICD_DGNS_CD6", "CLM_POA_IND_SW6", "ICD_DGNS_CD7", "CLM_POA_IND_SW7", "ICD_DGNS_CD8", "CLM_POA_IND_SW8", "ICD_DGNS_CD9", "CLM_POA_IND_SW9", "ICD_DGNS_CD10", "CLM_POA_IND_SW10", "ICD_DGNS_CD11", "CLM_POA_IND_SW11", "ICD_DGNS_CD12", "CLM_POA_IND_SW12", "ICD_DGNS_CD13", "CLM_POA_IND_SW13", "ICD_DGNS_CD14", "CLM_POA_IND_SW14", "ICD_DGNS_CD15", "CLM_POA_IND_SW15", "ICD_DGNS_CD16", "CLM_POA_IND_SW16", "ICD_DGNS_CD17", "CLM_POA_IND_SW17", "ICD_DGNS_CD18", "CLM_POA_IND_SW18", "ICD_DGNS_CD19", "CLM_POA_IND_SW19", "ICD_DGNS_CD20", "CLM_POA_IND_SW20", "ICD_DGNS_CD21", "CLM_POA_IND_SW21", "ICD_DGNS_CD22", "CLM_POA_IND_SW22", "ICD_DGNS_CD23", "CLM_POA_IND_SW23", "ICD_DGNS_CD24", "CLM_POA_IND_SW24", "ICD_DGNS_CD25", "CLM_POA_IND_SW25", "FST_DGNS_E_CD", "ICD_DGNS_E_CD1", "CLM_E_POA_IND_SW1", "ICD_DGNS_E_CD2", "CLM_E_POA_IND_SW2", "ICD_DGNS_E_CD3", "CLM_E_POA_IND_SW3", "ICD_DGNS_E_CD4", "CLM_E_POA_IND_SW4", "ICD_DGNS_E_CD5", "CLM_E_POA_IND_SW5", "ICD_DGNS_E_CD6", "CLM_E_POA_IND_SW6", "ICD_DGNS_E_CD7", "CLM_E_POA_IND_SW7", "ICD_DGNS_E_CD8", "CLM_E_POA_IND_SW8", "ICD_DGNS_E_CD9", "CLM_E_POA_IND_SW9", "ICD_DGNS_E_CD10", "CLM_E_POA_IND_SW10", "ICD_DGNS_E_CD11", "CLM_E_POA_IND_SW11", "ICD_DGNS_E_CD12", "CLM_E_POA_IND_SW12", "ICD_PRCDR_CD1", "PRCDR_DT1", "ICD_PRCDR_CD2", "PRCDR_DT2", "ICD_PRCDR_CD3", "PRCDR_DT3", "ICD_PRCDR_CD4", "PRCDR_DT4", "ICD_PRCDR_CD5", "PRCDR_DT5", "ICD_PRCDR_CD6", "PRCDR_DT6", "ICD_PRCDR_CD7", "PRCDR_DT7", "ICD_PRCDR_CD8", "PRCDR_DT8", "ICD_PRCDR_CD9", "PRCDR_DT9", "ICD_PRCDR_CD10", "PRCDR_DT10", "ICD_PRCDR_CD11", "PRCDR_DT11", "ICD_PRCDR_CD12", "PRCDR_DT12", "ICD_PRCDR_CD13", "PRCDR_DT13", "ICD_PRCDR_CD14", "PRCDR_DT14", "ICD_PRCDR_CD15", "PRCDR_DT15", "ICD_PRCDR_CD16", "PRCDR_DT16", "ICD_PRCDR_CD17", "PRCDR_DT17", "ICD_PRCDR_CD18", "PRCDR_DT18", "ICD_PRCDR_CD19", "PRCDR_DT19", "ICD_PRCDR_CD20", "PRCDR_DT20", "ICD_PRCDR_CD21", "PRCDR_DT21", "ICD_PRCDR_CD22", "PRCDR_DT22", "ICD_PRCDR_CD23", "PRCDR_DT23", "ICD_PRCDR_CD24", "PRCDR_DT24", "ICD_PRCDR_CD25", "PRCDR_DT25", "DOB_DT", "GNDR_CD", "BENE_RACE_CD", "BENE_CNTY_CD", "BENE_STATE_CD", "CWF_BENE_MDCR_STUS_CD", "CLM_TRTMT_AUTHRZTN_NUM", "CLM_PRCR_RTRN_CD", "CLM_IP_LOW_VOL_PMT_AMT", "CLM_CARE_IMPRVMT_MODEL_CD1", "CLM_CARE_IMPRVMT_MODEL_CD2", "CLM_CARE_IMPRVMT_MODEL_CD3", "CLM_CARE_IMPRVMT_MODEL_CD4", "CLM_BNDLD_MODEL_1_DSCNT_PCT", "CLM_BASE_OPRTG_DRG_AMT", "CLM_VBP_PRTCPNT_IND_CD", "CLM_VBP_ADJSTMT_PCT", "CLM_HRR_PRTCPNT_IND_CD", "CLM_HRR_ADJSTMT_PCT", "CLM_MODEL_4_READMSN_IND_CD", "CLM_UNCOMPD_CARE_PMT_AMT", "CLM_BNDLD_ADJSTMT_PMT_AMT", "CLM_VBP_ADJSTMT_PMT_AMT", "CLM_HRR_ADJSTMT_PMT_AMT", "EHR_PYMT_ADJSTMT_AMT", "PPS_STD_VAL_PYMT_AMT", "FINL_STD_AMT", "HAC_PGM_RDCTN_IND_SW", "EHR_PGM_RDCTN_IND_SW", "CLM_SITE_NTRL_PYMT_CST_AMT", "CLM_SITE_NTRL_PYMT_IPPS_AMT", "CLM_FULL_STD_PYMT_AMT", "CLM_SS_OUTLIER_STD_PYMT_AMT", "CLM_NEXT_GNRTN_ACO_IND_CD1", "CLM_NEXT_GNRTN_ACO_IND_CD2", "CLM_NEXT_GNRTN_ACO_IND_CD3", "CLM_NEXT_GNRTN_ACO_IND_CD4", "CLM_NEXT_GNRTN_ACO_IND_CD5", "ACO_ID_NUM")

colnames_claim_outpatient_2011_2015 <-
  c("DESY_SORT_KEY", "CLAIM_NO", "PRVDR_NUM", "CLM_THRU_DT", "NCH_NEAR_LINE_REC_IDENT_CD", "NCH_CLM_TYPE_CD", "CLAIM_QUERY_CODE", "CLM_FAC_TYPE_CD", "CLM_SRVC_CLSFCTN_TYPE_CD", "CLM_FREQ_CD", "FI_NUM", "CLM_MDCR_NON_PMT_RSN_CD", "CLM_PMT_AMT", "NCH_PRMRY_PYR_CLM_PD_AMT", "NCH_PRMRY_PYR_CD", "PRVDR_STATE_CD", "ORG_NPI_NUM", "AT_PHYSN_UPIN", "AT_PHYSN_NPI", "OP_PHYSN_UPIN", "OP_PHYSN_NPI", "OT_PHYSN_UPIN", "OT_PHYSN_NPI", "CLM_MCO_PD_SW", "PTNT_DSCHRG_STUS_CD", "CLM_TOT_CHRG_AMT", "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM", "NCH_PROFNL_CMPNT_CHRG_AMT", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12", "ICD_DGNS_CD13", "ICD_DGNS_VRSN_CD13", "ICD_DGNS_CD14", "ICD_DGNS_VRSN_CD14", "ICD_DGNS_CD15", "ICD_DGNS_VRSN_CD15", "ICD_DGNS_CD16", "ICD_DGNS_VRSN_CD16", "ICD_DGNS_CD17", "ICD_DGNS_VRSN_CD17", "ICD_DGNS_CD18", "ICD_DGNS_VRSN_CD18", "ICD_DGNS_CD19", "ICD_DGNS_VRSN_CD19", "ICD_DGNS_CD20", "ICD_DGNS_VRSN_CD20", "ICD_DGNS_CD21", "ICD_DGNS_VRSN_CD21", "ICD_DGNS_CD22", "ICD_DGNS_VRSN_CD22", "ICD_DGNS_CD23", "ICD_DGNS_VRSN_CD23", "ICD_DGNS_CD24", "ICD_DGNS_VRSN_CD24", "ICD_DGNS_CD25", "ICD_DGNS_VRSN_CD25", "FST_DGNS_E_CD", "FST_DGNS_E_VRSN_CD", "ICD_DGNS_E_CD1", "ICD_DGNS_E_VRSN_CD1", "ICD_DGNS_E_CD2", "ICD_DGNS_E_VRSN_CD2", "ICD_DGNS_E_CD3", "ICD_DGNS_E_VRSN_CD3", "ICD_DGNS_E_CD4", "ICD_DGNS_E_VRSN_CD4", "ICD_DGNS_E_CD5", "ICD_DGNS_E_VRSN_CD5", "ICD_DGNS_E_CD6", "ICD_DGNS_E_VRSN_CD6", "ICD_DGNS_E_CD7", "ICD_DGNS_E_VRSN_CD7", "ICD_DGNS_E_CD8", "ICD_DGNS_E_VRSN_CD8", "ICD_DGNS_E_CD9", "ICD_DGNS_E_VRSN_CD9", "ICD_DGNS_E_CD10", "ICD_DGNS_E_VRSN_CD10", "ICD_DGNS_E_CD11", "ICD_DGNS_E_VRSN_CD11", "ICD_DGNS_E_CD12", "ICD_DGNS_E_VRSN_CD12", "ICD_PRCDR_CD1", "ICD_PRCDR_VRSN_CD1", "PRCDR_DT1", "ICD_PRCDR_CD2", "ICD_PRCDR_VRSN_CD2", "PRCDR_DT2", "ICD_PRCDR_CD3", "ICD_PRCDR_VRSN_CD3", "PRCDR_DT3", "ICD_PRCDR_CD4", "ICD_PRCDR_VRSN_CD4", "PRCDR_DT4", "ICD_PRCDR_CD5", "ICD_PRCDR_VRSN_CD5", "PRCDR_DT5", "ICD_PRCDR_CD6", "ICD_PRCDR_VRSN_CD6", "PRCDR_DT6", "ICD_PRCDR_CD7", "ICD_PRCDR_VRSN_CD7", "PRCDR_DT7", "ICD_PRCDR_CD8", "ICD_PRCDR_VRSN_CD8", "PRCDR_DT8", "ICD_PRCDR_CD9", "ICD_PRCDR_VRSN_CD9", "PRCDR_DT9", "ICD_PRCDR_CD10", "ICD_PRCDR_VRSN_CD10", "PRCDR_DT10", "ICD_PRCDR_CD11", "ICD_PRCDR_VRSN_CD11", "PRCDR_DT11", "ICD_PRCDR_CD12", "ICD_PRCDR_VRSN_CD12", "PRCDR_DT12", "ICD_PRCDR_CD13", "ICD_PRCDR_VRSN_CD13", "PRCDR_DT13", "ICD_PRCDR_CD14", "ICD_PRCDR_VRSN_CD14", "PRCDR_DT14", "ICD_PRCDR_CD15", "ICD_PRCDR_VRSN_CD15", "PRCDR_DT15", "ICD_PRCDR_CD16", "ICD_PRCDR_VRSN_CD16", "PRCDR_DT16", "ICD_PRCDR_CD17", "ICD_PRCDR_VRSN_CD17", "PRCDR_DT17", "ICD_PRCDR_CD18", "ICD_PRCDR_VRSN_CD18", "PRCDR_DT18", "ICD_PRCDR_CD19", "ICD_PRCDR_VRSN_CD19", "PRCDR_DT19", "ICD_PRCDR_CD20", "ICD_PRCDR_VRSN_CD20", "PRCDR_DT20", "ICD_PRCDR_CD21", "ICD_PRCDR_VRSN_CD21", "PRCDR_DT21", "ICD_PRCDR_CD22", "ICD_PRCDR_VRSN_CD22", "PRCDR_DT22", "ICD_PRCDR_CD23", "ICD_PRCDR_VRSN_CD23", "PRCDR_DT23", "ICD_PRCDR_CD24", "ICD_PRCDR_VRSN_CD24", "PRCDR_DT24", "ICD_PRCDR_CD25", "ICD_PRCDR_VRSN_CD25", "PRCDR_DT25", "RSN_VISIT_CD1", "RSN_VISIT_VRSN_CD1", "RSN_VISIT_CD2", "RSN_VISIT_VRSN_CD2", "RSN_VISIT_CD3", "RSN_VISIT_VRSN_CD3", "NCH_BENE_PTB_DDCTBL_AMT", "NCH_BENE_PTB_COINSRNC_AMT", "CLM_OP_PRVDR_PMT_AMT", "CLM_OP_BENE_PMT_AMT", "DOB_DT", "GNDR_CD", "BENE_RACE_CD", "BENE_CNTY_CD", "BENE_STATE_CD", "CWF_BENE_MDCR_STUS_CD", "FI_CLM_ACTN_CD")

colnames_claim_outpatient_2016_2020 <-
  c("DESY_SORT_KEY", "CLAIM_NO", "PRVDR_NUM", "CLM_THRU_DT", "NCH_NEAR_LINE_REC_IDENT_CD", "NCH_CLM_TYPE_CD", "CLAIM_QUERY_CODE", "CLM_FAC_TYPE_CD", "CLM_SRVC_CLSFCTN_TYPE_CD", "CLM_FREQ_CD", "FI_NUM", "CLM_MDCR_NON_PMT_RSN_CD", "CLM_PMT_AMT", "NCH_PRMRY_PYR_CLM_PD_AMT", "NCH_PRMRY_PYR_CD", "PRVDR_STATE_CD", "ORG_NPI_NUM", "SRVC_LOC_NPI_NUM", "AT_PHYSN_UPIN", "AT_PHYSN_NPI", "AT_PHYSN_SPCLTY_CD", "OP_PHYSN_UPIN", "OP_PHYSN_NPI", "OP_PHYSN_SPCLTY_CD", "OT_PHYSN_UPIN", "OT_PHYSN_NPI", "OT_PHYSN_SPCLTY_CD", "RNDRNG_PHYSN_NPI", "RNDRNG_PHYSN_SPCLTY_CD", "RFR_PHYSN_NPI", "RFR_PHYSN_SPCLTY_CD", "CLM_MCO_PD_SW", "PTNT_DSCHRG_STUS_CD", "CLM_TOT_CHRG_AMT", "NCH_BENE_BLOOD_DDCTBL_LBLTY_AM", "NCH_PROFNL_CMPNT_CHRG_AMT", "PRNCPAL_DGNS_CD", "ICD_DGNS_CD1", "ICD_DGNS_CD2", "ICD_DGNS_CD3", "ICD_DGNS_CD4", "ICD_DGNS_CD5", "ICD_DGNS_CD6", "ICD_DGNS_CD7", "ICD_DGNS_CD8", "ICD_DGNS_CD9", "ICD_DGNS_CD10", "ICD_DGNS_CD11", "ICD_DGNS_CD12", "ICD_DGNS_CD13", "ICD_DGNS_CD14", "ICD_DGNS_CD15", "ICD_DGNS_CD16", "ICD_DGNS_CD17", "ICD_DGNS_CD18", "ICD_DGNS_CD19", "ICD_DGNS_CD20", "ICD_DGNS_CD21", "ICD_DGNS_CD22", "ICD_DGNS_CD23", "ICD_DGNS_CD24", "ICD_DGNS_CD25", "FST_DGNS_E_CD", "ICD_DGNS_E_CD1", "ICD_DGNS_E_CD2", "ICD_DGNS_E_CD3", "ICD_DGNS_E_CD4", "ICD_DGNS_E_CD5", "ICD_DGNS_E_CD6", "ICD_DGNS_E_CD7", "ICD_DGNS_E_CD8", "ICD_DGNS_E_CD9", "ICD_DGNS_E_CD10", "ICD_DGNS_E_CD11", "ICD_DGNS_E_CD12", "ICD_PRCDR_CD1", "PRCDR_DT1", "ICD_PRCDR_CD2", "PRCDR_DT2", "ICD_PRCDR_CD3", "PRCDR_DT3", "ICD_PRCDR_CD4", "PRCDR_DT4", "ICD_PRCDR_CD5", "PRCDR_DT5", "ICD_PRCDR_CD6", "PRCDR_DT6", "ICD_PRCDR_CD7", "PRCDR_DT7", "ICD_PRCDR_CD8", "PRCDR_DT8", "ICD_PRCDR_CD9", "PRCDR_DT9", "ICD_PRCDR_CD10", "PRCDR_DT10", "ICD_PRCDR_CD11", "PRCDR_DT11", "ICD_PRCDR_CD12", "PRCDR_DT12", "ICD_PRCDR_CD13", "PRCDR_DT13", "ICD_PRCDR_CD14", "PRCDR_DT14", "ICD_PRCDR_CD15", "PRCDR_DT15", "ICD_PRCDR_CD16", "PRCDR_DT16", "ICD_PRCDR_CD17", "PRCDR_DT17", "ICD_PRCDR_CD18", "PRCDR_DT18", "ICD_PRCDR_CD19", "PRCDR_DT19", "ICD_PRCDR_CD20", "PRCDR_DT20", "ICD_PRCDR_CD21", "PRCDR_DT21", "ICD_PRCDR_CD22", "PRCDR_DT22", "ICD_PRCDR_CD23", "PRCDR_DT23", "ICD_PRCDR_CD24", "PRCDR_DT24", "ICD_PRCDR_CD25", "PRCDR_DT25", "RSN_VISIT_CD1", "RSN_VISIT_CD2", "RSN_VISIT_CD3", "NCH_BENE_PTB_DDCTBL_AMT", "NCH_BENE_PTB_COINSRNC_AMT", "CLM_OP_PRVDR_PMT_AMT", "CLM_OP_BENE_PMT_AMT", "DOB_DT", "GNDR_CD", "BENE_RACE_CD", "BENE_CNTY_CD", "BENE_STATE_CD", "CWF_BENE_MDCR_STUS_CD", "FI_CLM_ACTN_CD", "NCH_BLOOD_PNTS_FRNSHD_QTY", "CLM_TRTMT_AUTHRZTN_NUM", "CLM_PRCR_RTRN_CD", "CLM_OP_TRANS_TYPE_CD", "CLM_OP_ESRD_MTHD_CD", "CLM_NEXT_GNRTN_ACO_IND_CD1", "CLM_NEXT_GNRTN_ACO_IND_CD2", "CLM_NEXT_GNRTN_ACO_IND_CD3", "CLM_NEXT_GNRTN_ACO_IND_CD4", "CLM_NEXT_GNRTN_ACO_IND_CD5", "ACO_ID_NUM")

In [ ]:
colnames(claim_carrier_2013) <- colnames_claim_carrier_2011_2015
colnames(claim_carrier_2014) <- colnames_claim_carrier_2011_2015
colnames(claim_carrier_2015) <- colnames_claim_carrier_2011_2015
colnames(claim_carrier_2016) <- colnames_claim_carrier_2016_2020
colnames(claim_carrier_2017) <- colnames_claim_carrier_2016_2020
colnames(claim_carrier_2018) <- colnames_claim_carrier_2016_2020
colnames(claim_carrier_2019) <- colnames_claim_carrier_2016_2020
colnames(claim_carrier_2020) <- colnames_claim_carrier_2016_2020


colnames(claim_inpatient_2013) <- colnames_claim_inpatient_2011_2015
colnames(claim_inpatient_2014) <- colnames_claim_inpatient_2011_2015
colnames(claim_inpatient_2015) <- colnames_claim_inpatient_2011_2015
colnames(claim_inpatient_2016) <- colnames_claim_inpatient_2016_2020
colnames(claim_inpatient_2017) <- colnames_claim_inpatient_2016_2020
colnames(claim_inpatient_2018) <- colnames_claim_inpatient_2016_2020
colnames(claim_inpatient_2019) <- colnames_claim_inpatient_2016_2020
colnames(claim_inpatient_2020) <- colnames_claim_inpatient_2016_2020


colnames(claim_outpatient_2013) <- colnames_claim_outpatient_2011_2015
colnames(claim_outpatient_2014) <- colnames_claim_outpatient_2011_2015
colnames(claim_outpatient_2015) <- colnames_claim_outpatient_2011_2015
colnames(claim_outpatient_2016) <- colnames_claim_outpatient_2016_2020
colnames(claim_outpatient_2017) <- colnames_claim_outpatient_2016_2020
colnames(claim_outpatient_2018) <- colnames_claim_outpatient_2016_2020
colnames(claim_outpatient_2019) <- colnames_claim_outpatient_2016_2020
colnames(claim_outpatient_2020) <- colnames_claim_outpatient_2016_2020

In [ ]:
claim_inpatient_2016[,":="(ADMTG_DGNS_VRSN_CD =0, PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_inpatient_2017[,":="(ADMTG_DGNS_VRSN_CD =0, PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_inpatient_2018[,":="(ADMTG_DGNS_VRSN_CD =0, PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_inpatient_2019[,":="(ADMTG_DGNS_VRSN_CD =0, PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_inpatient_2020[,":="(ADMTG_DGNS_VRSN_CD =0, PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

In [ ]:


claim_outpatient_2016[,":="(PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_outpatient_2017[,":="( PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_outpatient_2018[,":="( PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_outpatient_2019[,":="( PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]

claim_outpatient_2020[,":="( PRNCPAL_DGNS_VRSN_CD=0, ICD_DGNS_VRSN_CD1=0, ICD_DGNS_VRSN_CD2=0, ICD_DGNS_VRSN_CD3=0, ICD_DGNS_VRSN_CD4=0, ICD_DGNS_VRSN_CD5=0, ICD_DGNS_VRSN_CD6=0, ICD_DGNS_VRSN_CD7=0, ICD_DGNS_VRSN_CD8=0, ICD_DGNS_VRSN_CD9=0, ICD_DGNS_VRSN_CD10=0, ICD_DGNS_VRSN_CD11=0, ICD_DGNS_VRSN_CD12=0, ICD_DGNS_VRSN_CD13=0, ICD_DGNS_VRSN_CD14=0, ICD_DGNS_VRSN_CD15=0, ICD_DGNS_VRSN_CD16=0, ICD_DGNS_VRSN_CD17=0, ICD_DGNS_VRSN_CD18=0, ICD_DGNS_VRSN_CD19=0, ICD_DGNS_VRSN_CD20=0, ICD_DGNS_VRSN_CD21=0, ICD_DGNS_VRSN_CD22=0, ICD_DGNS_VRSN_CD23=0, ICD_DGNS_VRSN_CD24=0, ICD_DGNS_VRSN_CD25=0)]


In [ ]:
head(claim_outpatient_2016)

DESY_SORT_KEY,CLAIM_NO,PRVDR_NUM,CLM_THRU_DT,NCH_NEAR_LINE_REC_IDENT_CD,NCH_CLM_TYPE_CD,CLAIM_QUERY_CODE,CLM_FAC_TYPE_CD,CLM_SRVC_CLSFCTN_TYPE_CD,CLM_FREQ_CD,FI_NUM,CLM_MDCR_NON_PMT_RSN_CD,CLM_PMT_AMT,NCH_PRMRY_PYR_CLM_PD_AMT,NCH_PRMRY_PYR_CD,PRVDR_STATE_CD,ORG_NPI_NUM,SRVC_LOC_NPI_NUM,AT_PHYSN_UPIN,AT_PHYSN_NPI,AT_PHYSN_SPCLTY_CD,OP_PHYSN_UPIN,OP_PHYSN_NPI,OP_PHYSN_SPCLTY_CD,OT_PHYSN_UPIN,OT_PHYSN_NPI,OT_PHYSN_SPCLTY_CD,RNDRNG_PHYSN_NPI,RNDRNG_PHYSN_SPCLTY_CD,RFR_PHYSN_NPI,RFR_PHYSN_SPCLTY_CD,CLM_MCO_PD_SW,PTNT_DSCHRG_STUS_CD,CLM_TOT_CHRG_AMT,NCH_BENE_BLOOD_DDCTBL_LBLTY_AM,NCH_PROFNL_CMPNT_CHRG_AMT,PRNCPAL_DGNS_CD,ICD_DGNS_CD1,ICD_DGNS_CD2,ICD_DGNS_CD3,ICD_DGNS_CD4,ICD_DGNS_CD5,ICD_DGNS_CD6,ICD_DGNS_CD7,ICD_DGNS_CD8,ICD_DGNS_CD9,ICD_DGNS_CD10,ICD_DGNS_CD11,ICD_DGNS_CD12,ICD_DGNS_CD13,ICD_DGNS_CD14,ICD_DGNS_CD15,ICD_DGNS_CD16,ICD_DGNS_CD17,ICD_DGNS_CD18,ICD_DGNS_CD19,ICD_DGNS_CD20,ICD_DGNS_CD21,ICD_DGNS_CD22,ICD_DGNS_CD23,ICD_DGNS_CD24,ICD_DGNS_CD25,FST_DGNS_E_CD,ICD_DGNS_E_CD1,ICD_DGNS_E_CD2,ICD_DGNS_E_CD3,ICD_DGNS_E_CD4,ICD_DGNS_E_CD5,ICD_DGNS_E_CD6,ICD_DGNS_E_CD7,ICD_DGNS_E_CD8,ICD_DGNS_E_CD9,ICD_DGNS_E_CD10,ICD_DGNS_E_CD11,ICD_DGNS_E_CD12,ICD_PRCDR_CD1,PRCDR_DT1,ICD_PRCDR_CD2,PRCDR_DT2,ICD_PRCDR_CD3,PRCDR_DT3,ICD_PRCDR_CD4,PRCDR_DT4,ICD_PRCDR_CD5,PRCDR_DT5,ICD_PRCDR_CD6,PRCDR_DT6,ICD_PRCDR_CD7,PRCDR_DT7,ICD_PRCDR_CD8,PRCDR_DT8,ICD_PRCDR_CD9,PRCDR_DT9,ICD_PRCDR_CD10,PRCDR_DT10,ICD_PRCDR_CD11,PRCDR_DT11,ICD_PRCDR_CD12,PRCDR_DT12,ICD_PRCDR_CD13,PRCDR_DT13,ICD_PRCDR_CD14,PRCDR_DT14,ICD_PRCDR_CD15,PRCDR_DT15,ICD_PRCDR_CD16,PRCDR_DT16,ICD_PRCDR_CD17,PRCDR_DT17,ICD_PRCDR_CD18,PRCDR_DT18,ICD_PRCDR_CD19,PRCDR_DT19,ICD_PRCDR_CD20,PRCDR_DT20,ICD_PRCDR_CD21,PRCDR_DT21,ICD_PRCDR_CD22,PRCDR_DT22,ICD_PRCDR_CD23,PRCDR_DT23,ICD_PRCDR_CD24,PRCDR_DT24,ICD_PRCDR_CD25,PRCDR_DT25,RSN_VISIT_CD1,RSN_VISIT_CD2,RSN_VISIT_CD3,NCH_BENE_PTB_DDCTBL_AMT,NCH_BENE_PTB_COINSRNC_AMT,CLM_OP_PRVDR_PMT_AMT,CLM_OP_BENE_PMT_AMT,DOB_DT,GNDR_CD,BENE_RACE_CD,BENE_CNTY_CD,BENE_STATE_CD,CWF_BENE_MDCR_STUS_CD,FI_CLM_ACTN_CD,NCH_BLOOD_PNTS_FRNSHD_QTY,CLM_TRTMT_AUTHRZTN_NUM,CLM_PRCR_RTRN_CD,CLM_OP_TRANS_TYPE_CD,CLM_OP_ESRD_MTHD_CD,CLM_NEXT_GNRTN_ACO_IND_CD1,CLM_NEXT_GNRTN_ACO_IND_CD2,CLM_NEXT_GNRTN_ACO_IND_CD3,CLM_NEXT_GNRTN_ACO_IND_CD4,CLM_NEXT_GNRTN_ACO_IND_CD5,ACO_ID_NUM,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_VRSN_CD1,ICD_DGNS_VRSN_CD2,ICD_DGNS_VRSN_CD3,ICD_DGNS_VRSN_CD4,ICD_DGNS_VRSN_CD5,ICD_DGNS_VRSN_CD6,ICD_DGNS_VRSN_CD7,ICD_DGNS_VRSN_CD8,ICD_DGNS_VRSN_CD9,ICD_DGNS_VRSN_CD10,ICD_DGNS_VRSN_CD11,ICD_DGNS_VRSN_CD12,ICD_DGNS_VRSN_CD13,ICD_DGNS_VRSN_CD14,ICD_DGNS_VRSN_CD15,ICD_DGNS_VRSN_CD16,ICD_DGNS_VRSN_CD17,ICD_DGNS_VRSN_CD18,ICD_DGNS_VRSN_CD19,ICD_DGNS_VRSN_CD20,ICD_DGNS_VRSN_CD21,ICD_DGNS_VRSN_CD22,ICD_DGNS_VRSN_CD23,ICD_DGNS_VRSN_CD24,ICD_DGNS_VRSN_CD25
<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<chr>,<lgl>,<dbl>,<chr>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1e+08,2,220071,20160119,W,40,3,1,3,1,14211,NA,104.26,0,NA,22,1023049236,NA,NA,1174560288,11,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,01,846.02,0,NA,J449,J449,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,

In [ ]:
choose_columns <- function(data_list, columns) {
  require(data.table)
  data_list <- lapply(data_list, function(data) {
    data[, ..columns]
  })
  result <- rbindlist(data_list)
  return(result)
}

claim_carrier_all_years <- choose_columns(
  list(
    claim_carrier_2013,
    claim_carrier_2014,
    claim_carrier_2015,
    claim_carrier_2016,
    claim_carrier_2017,
    claim_carrier_2018,
    claim_carrier_2019,
    claim_carrier_2020
  ),
  columns = c("DESY_SORT_KEY", "CLAIM_NO", "CLM_THRU_DT", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12")
)

claim_carrier_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
claim_carrier_all_years[, month_year := format(date, "%Y-%m")]
claim_carrier_all_years[, DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]


claim_inpatient_all_years <- choose_columns(
  list(
    claim_inpatient_2013,
    claim_inpatient_2014,
    claim_inpatient_2015,
    claim_inpatient_2016,
    claim_inpatient_2017,
    claim_inpatient_2018,
    claim_inpatient_2019,
    claim_inpatient_2020
  ),
  columns = c("DESY_SORT_KEY", "CLAIM_NO", "CLM_THRU_DT", "ADMTG_DGNS_CD", "ADMTG_DGNS_VRSN_CD", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "CLM_POA_IND_SW1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "CLM_POA_IND_SW2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "CLM_POA_IND_SW3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "CLM_POA_IND_SW4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "CLM_POA_IND_SW5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "CLM_POA_IND_SW6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "CLM_POA_IND_SW7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "CLM_POA_IND_SW8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "CLM_POA_IND_SW9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "CLM_POA_IND_SW10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "CLM_POA_IND_SW11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12", "CLM_POA_IND_SW12", "ICD_DGNS_CD13", "ICD_DGNS_VRSN_CD13", "CLM_POA_IND_SW13", "ICD_DGNS_CD14", "ICD_DGNS_VRSN_CD14", "CLM_POA_IND_SW14", "ICD_DGNS_CD15", "ICD_DGNS_VRSN_CD15", "CLM_POA_IND_SW15", "ICD_DGNS_CD16", "ICD_DGNS_VRSN_CD16", "CLM_POA_IND_SW16", "ICD_DGNS_CD17", "ICD_DGNS_VRSN_CD17", "CLM_POA_IND_SW17", "ICD_DGNS_CD18", "ICD_DGNS_VRSN_CD18", "CLM_POA_IND_SW18", "ICD_DGNS_CD19", "ICD_DGNS_VRSN_CD19", "CLM_POA_IND_SW19", "ICD_DGNS_CD20", "ICD_DGNS_VRSN_CD20", "CLM_POA_IND_SW20", "ICD_DGNS_CD21", "ICD_DGNS_VRSN_CD21", "CLM_POA_IND_SW21", "ICD_DGNS_CD22", "ICD_DGNS_VRSN_CD22", "CLM_POA_IND_SW22", "ICD_DGNS_CD23", "ICD_DGNS_VRSN_CD23", "CLM_POA_IND_SW23", "ICD_DGNS_CD24", "ICD_DGNS_VRSN_CD24", "CLM_POA_IND_SW24", "ICD_DGNS_CD25", "ICD_DGNS_VRSN_CD25", "CLM_POA_IND_SW25")
)

claim_inpatient_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
claim_inpatient_all_years[, month_year := format(date, "%Y-%m")]
claim_inpatient_all_years[, DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]


claim_outpatient_all_years <- choose_columns(
  list(
    claim_outpatient_2013,
    claim_outpatient_2014,
    claim_outpatient_2015,
    claim_outpatient_2016,
    claim_outpatient_2017,
    claim_outpatient_2018,
    claim_outpatient_2019,
    claim_outpatient_2020
  ),
  columns = c("DESY_SORT_KEY", "CLAIM_NO", "CLM_THRU_DT", "PRNCPAL_DGNS_CD", "PRNCPAL_DGNS_VRSN_CD", "ICD_DGNS_CD1", "ICD_DGNS_VRSN_CD1", "ICD_DGNS_CD2", "ICD_DGNS_VRSN_CD2", "ICD_DGNS_CD3", "ICD_DGNS_VRSN_CD3", "ICD_DGNS_CD4", "ICD_DGNS_VRSN_CD4", "ICD_DGNS_CD5", "ICD_DGNS_VRSN_CD5", "ICD_DGNS_CD6", "ICD_DGNS_VRSN_CD6", "ICD_DGNS_CD7", "ICD_DGNS_VRSN_CD7", "ICD_DGNS_CD8", "ICD_DGNS_VRSN_CD8", "ICD_DGNS_CD9", "ICD_DGNS_VRSN_CD9", "ICD_DGNS_CD10", "ICD_DGNS_VRSN_CD10", "ICD_DGNS_CD11", "ICD_DGNS_VRSN_CD11", "ICD_DGNS_CD12", "ICD_DGNS_VRSN_CD12", "ICD_DGNS_CD13", "ICD_DGNS_VRSN_CD13", "ICD_DGNS_CD14", "ICD_DGNS_VRSN_CD14", "ICD_DGNS_CD15", "ICD_DGNS_VRSN_CD15", "ICD_DGNS_CD16", "ICD_DGNS_VRSN_CD16", "ICD_DGNS_CD17", "ICD_DGNS_VRSN_CD17", "ICD_DGNS_CD18", "ICD_DGNS_VRSN_CD18", "ICD_DGNS_CD19", "ICD_DGNS_VRSN_CD19", "ICD_DGNS_CD20", "ICD_DGNS_VRSN_CD20", "ICD_DGNS_CD21", "ICD_DGNS_VRSN_CD21", "ICD_DGNS_CD22", "ICD_DGNS_VRSN_CD22", "ICD_DGNS_CD23", "ICD_DGNS_VRSN_CD23", "ICD_DGNS_CD24", "ICD_DGNS_VRSN_CD24", "ICD_DGNS_CD25", "ICD_DGNS_VRSN_CD25")
)

claim_outpatient_all_years[, date := as.IDate(as.character(CLM_THRU_DT), "%Y%m%d")][order(date)]
claim_outpatient_all_years[, month_year := format(date, "%Y-%m")]
claim_outpatient_all_years[, DESY_SORT_KEY := as.integer(DESY_SORT_KEY)]

In [ ]:
claim_carrier_all_years[, year := format(date, "%Y")]

claim_inpatient_all_years[, year := format(date, "%Y")]

claim_outpatient_all_years[, year := format(date, "%Y")]

In [ ]:
write_fst(claim_carrier_all_years, "/work/postresearch/Shared/Projects/Data_fst/claim_carrier_all_years.fst")
write_fst(claim_inpatient_all_years, "/work/postresearch/Shared/Projects/Data_fst/claim_inpatient_all_years.fst")
write_fst(claim_outpatient_all_years, "/work/postresearch/Shared/Projects/Data_fst/claim_outpatient_all_years.fst")

## Reading fst claim level files

In [ ]:
claim_carrier_all_years=read_fst("/work/postresearch/Shared/Projects/Data_fst/claim_carrier_all_years.fst",as.data.table = T)
claim_inpatient_all_years=read_fst("/work/postresearch/Shared/Projects/Data_fst/claim_inpatient_all_years.fst",as.data.table = T)
claim_outpatient_all_years=read_fst("/work/postresearch/Shared/Projects/Data_fst/claim_outpatient_all_years.fst",as.data.table = T)

In [ ]:
head(claim_carrier_all_years)
head(claim_inpatient_all_years)
head(claim_outpatient_all_years)

DESY_SORT_KEY,CLAIM_NO,CLM_THRU_DT,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_CD1,ICD_DGNS_VRSN_CD1,ICD_DGNS_CD2,ICD_DGNS_VRSN_CD2,ICD_DGNS_CD3,ICD_DGNS_VRSN_CD3,ICD_DGNS_CD4,ICD_DGNS_VRSN_CD4,ICD_DGNS_CD5,ICD_DGNS_VRSN_CD5,ICD_DGNS_CD6,ICD_DGNS_VRSN_CD6,ICD_DGNS_CD7,ICD_DGNS_VRSN_CD7,ICD_DGNS_CD8,ICD_DGNS_VRSN_CD8,ICD_DGNS_CD9,ICD_DGNS_VRSN_CD9,ICD_DGNS_CD10,ICD_DGNS_VRSN_CD10,ICD_DGNS_CD11,ICD_DGNS_VRSN_CD11,ICD_DGNS_CD12,ICD_DGNS_VRSN_CD12,date,month_year,year
<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<date>,<chr>,<chr>
100000015,2,20130425,496,9,496,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-04-25,2013-04,2013
100000015,3,20130528,41400,9,41400,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-05-28,2013-05,2013
100000015,4,20130719,496,9,496,9,79319,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-07-19,2013-07,2013
100000015,5,20130719,496,9,496,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-07-19,2013-07,2013
100000015,6,20131021,41400,9,41400,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-10-21,2013-10,2013
100000015,7,20131105,79311,9,79311,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-11-05,2013-11,2013


DESY_SORT_KEY,CLAIM_NO,CLM_THRU_DT,ADMTG_DGNS_CD,ADMTG_DGNS_VRSN_CD,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_CD1,ICD_DGNS_VRSN_CD1,CLM_POA_IND_SW1,ICD_DGNS_CD2,ICD_DGNS_VRSN_CD2,CLM_POA_IND_SW2,ICD_DGNS_CD3,ICD_DGNS_VRSN_CD3,CLM_POA_IND_SW3,ICD_DGNS_CD4,ICD_DGNS_VRSN_CD4,CLM_POA_IND_SW4,ICD_DGNS_CD5,ICD_DGNS_VRSN_CD5,CLM_POA_IND_SW5,ICD_DGNS_CD6,ICD_DGNS_VRSN_CD6,CLM_POA_IND_SW6,ICD_DGNS_CD7,ICD_DGNS_VRSN_CD7,CLM_POA_IND_SW7,ICD_DGNS_CD8,ICD_DGNS_VRSN_CD8,CLM_POA_IND_SW8,ICD_DGNS_CD9,ICD_DGNS_VRSN_CD9,CLM_POA_IND_SW9,ICD_DGNS_CD10,ICD_DGNS_VRSN_CD10,CLM_POA_IND_SW10,ICD_DGNS_CD11,ICD_DGNS_VRSN_CD11,CLM_POA_IND_SW11,ICD_DGNS_CD12,ICD_DGNS_VRSN_CD12,CLM_POA_IND_SW12,ICD_DGNS_CD13,ICD_DGNS_VRSN_CD13,CLM_POA_IND_SW13,ICD_DGNS_CD14,ICD_DGNS_VRSN_CD14,CLM_POA_IND_SW14,ICD_DGNS_CD15,ICD_DGNS_VRSN_CD15,CLM_POA_IND_SW15,ICD_DGNS_CD16,ICD_DGNS_VRSN_CD16,CLM_POA_IND_SW16,ICD_DGNS_CD17,ICD_DGNS_VRSN_CD17,CLM_POA_IND_SW17,ICD_DGNS_CD18,ICD_DGNS_VRSN_CD18,CLM_POA_IND_SW18,ICD_DGNS_CD19,ICD_DGNS_VRSN_CD19,CLM_POA_IND_SW19,ICD_DGNS_CD20,ICD_DGNS_VRSN_CD20,CLM_POA_IND_SW20,ICD_DGNS_CD21,ICD_DGNS_VRSN_CD21,CLM_POA_IND_SW21,ICD_DGNS_CD22,ICD_DGNS_VRSN_CD22,CLM_POA_IND_SW22,ICD_DGNS_CD23,ICD_DGNS_VRSN_CD23,CLM_POA_IND_SW23,ICD_DGNS_CD24,ICD_DGNS_VRSN_CD24,CLM_POA_IND_SW24,ICD_DGNS_CD25,ICD_DGNS_VRSN_CD25,CLM_POA_IND_SW25,date,month_year,year
<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<date>,<chr>,<chr>
100000099,2,20131027,71536,9,71536,9,71536,9,Y,72700,9,Y,4019,9,Y,2731,9,Y,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-10-27,2013-10,2013
100000099,3,20131103,2859,9,99812,9,99812,9,Y,2761,9,Y,2800,9,Y,V4365,9,0,42731,9,Y,V5861,9,0,4019,9,Y,2724,9,Y,V173,9,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-11-03,2013-11,2013
100000315,4,20130103,56211,9,0090,9,0090,9,Y,56211,9,Y,5849,9,Y,27651,9,Y,2768,9,Y,4168,9,Y,43889,9,0,78093,9,Y,3004,9,Y,59651,9,Y,30000,9,Y,56400,9,Y,4241,9,Y,4280,9,Y,V4986,9,Y,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-01-03,2013-01,2013
100000559,5,20131224,78060,9,7907,9,7907,9,Y,5856,9,Y,99681,9,Y,40391,9,Y,28981,9,Y,58881,9,Y,75989,9,0,78060,9,Y,587,9,Y,28521,9,Y,34590,9,Y,5880,9,Y,2724,9,Y,V180,9,0,V1749,9,0,V1582,9,0,V1254,9,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-12-24,2013-12,2013
100000905,6,20130802,5990,9,5990,9,5990,9,Y,04185,9,Y,25000,9,Y,5920,9,Y,2449,9,Y,2252,9,Y,4019,9,Y,5935,9,Y,V5869,9,0,V1083,9,0,V140,9,0,V146,9,0,V1504,9,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-08-02,2013-08,2013
100001177,7,20130825,78559,9,1889,9,1889,9,Y,5693,9,Y,1970,9,Y,4260,9,Y,5849,9,Y,78559,9,Y,59984,9,Y,V4986,9,Y,7802,9,Y,5859,9,Y,40390,9,Y,25000,9,Y,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-08-25,2013-08,2013


DESY_SORT_KEY,CLAIM_NO,CLM_THRU_DT,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_CD1,ICD_DGNS_VRSN_CD1,ICD_DGNS_CD2,ICD_DGNS_VRSN_CD2,ICD_DGNS_CD3,ICD_DGNS_VRSN_CD3,ICD_DGNS_CD4,ICD_DGNS_VRSN_CD4,ICD_DGNS_CD5,ICD_DGNS_VRSN_CD5,ICD_DGNS_CD6,ICD_DGNS_VRSN_CD6,ICD_DGNS_CD7,ICD_DGNS_VRSN_CD7,ICD_DGNS_CD8,ICD_DGNS_VRSN_CD8,ICD_DGNS_CD9,ICD_DGNS_VRSN_CD9,ICD_DGNS_CD10,ICD_DGNS_VRSN_CD10,ICD_DGNS_CD11,ICD_DGNS_VRSN_CD11,ICD_DGNS_CD12,ICD_DGNS_VRSN_CD12,ICD_DGNS_CD13,ICD_DGNS_VRSN_CD13,ICD_DGNS_CD14,ICD_DGNS_VRSN_CD14,ICD_DGNS_CD15,ICD_DGNS_VRSN_CD15,ICD_DGNS_CD16,ICD_DGNS_VRSN_CD16,ICD_DGNS_CD17,ICD_DGNS_VRSN_CD17,ICD_DGNS_CD18,ICD_DGNS_VRSN_CD18,ICD_DGNS_CD19,ICD_DGNS_VRSN_CD19,ICD_DGNS_CD20,ICD_DGNS_VRSN_CD20,ICD_DGNS_CD21,ICD_DGNS_VRSN_CD21,ICD_DGNS_CD22,ICD_DGNS_VRSN_CD22,ICD_DGNS_CD23,ICD_DGNS_VRSN_CD23,ICD_DGNS_CD24,ICD_DGNS_VRSN_CD24,ICD_DGNS_CD25,ICD_DGNS_VRSN_CD25,date,month_year,year
<int>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<date>,<chr>,<chr>
100000015,2,20130425,496,9,496,9,7862,9,51889,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-04-25,2013-04,2013
100000015,3,20130719,496,9,496,9,79319,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-07-19,2013-07,2013
100000015,4,20131014,V0481,9,V0481,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-10-14,2013-10,2013
100000019,5,20130523,53510,9,53510,9,53010,9,V7651,9,1539,9,2809,9,56210,9,56400,9,4019,9,2720,9,V5866,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-05-23,2013-05,2013
100000019,6,20130529,1541,9,1541,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-05-29,2013-05,2013
100000019,7,20130603,1540,9,1540,9,7856,9,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2013-06-03,2013-06,2013


In [ ]:
claim_carrier_all_years=head(claim_carrier_all_years,100)
claim_inpatient_all_years=head(claim_inpatient_all_years,100)
claim_outpatient_all_years=head(claim_outpatient_all_years,100)

## Conditions

### Carrier

In [ ]:
conditions_claim_carrier=

claim_carrier_all_years %>%
summarise(
  
  DESY_SORT_KEY = DESY_SORT_KEY,
  
  year=year,
  
  date=date,
  
  month_year=month_year,
  
  
  PRNCPAL_DGNS_is_hypertension=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes)),
  
  
  DGNS_1_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_2_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% hypertension_icd_9_codes)),

  DGNS_3_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_4_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_6_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_7_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_8_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_9_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_10_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_11_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_12_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% hypertension_icd_9_codes)),
  
  PRNCPAL_DGNS_is_arthritis=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_1_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_2_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% arthritis_icd_9_codes)),

  DGNS_3_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_4_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_6_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_7_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_8_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_9_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_10_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_11_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_12_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% arthritis_icd_9_codes)),
  
  PRNCPAL_DGNS_is_IHD=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% IHD_icd_9_codes)),
  
    DGNS_1_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_2_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% IHD_icd_9_codes)),

  DGNS_3_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_4_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_6_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_7_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_8_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_9_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_10_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_11_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_12_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% IHD_icd_9_codes)),
  
  PRNCPAL_DGNS_is_diabetes=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_1_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_9_codes)),

  DGNS_1_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_2_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% diabetes_icd_9_codes)),

  DGNS_3_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_4_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_6_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_7_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_8_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_9_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_10_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_11_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_12_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% diabetes_icd_9_codes)),
  
  PRNCPAL_DGNS_is_depression=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 4) %in% depression_icd_9_codes)),
  
    DGNS_1_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_2_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 4) %in% depression_icd_9_codes)),

  DGNS_3_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_4_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_6_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_7_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_8_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_9_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_10_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_11_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_12_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 4) %in% depression_icd_9_codes))
)%>%
as.data.table()

conditions_claim_carrier[is.na(conditions_claim_carrier)] <- 0

conditions_claim_carrier=
conditions_claim_carrier%>%
summarise(
  
  DESY_SORT_KEY = DESY_SORT_KEY,
  
  year=year,
  
  date=date,
  
  month_year=month_year,
  

  is_hypertension=
  PRNCPAL_DGNS_is_hypertension | DGNS_1_is_hypertension | DGNS_2_is_hypertension | DGNS_3_is_hypertension | DGNS_4_is_hypertension | DGNS_6_is_hypertension | DGNS_7_is_hypertension | 
  DGNS_8_is_hypertension | DGNS_9_is_hypertension | DGNS_10_is_hypertension | DGNS_11_is_hypertension | DGNS_12_is_hypertension,
  
  is_arthritis=
  PRNCPAL_DGNS_is_arthritis | DGNS_1_is_arthritis | DGNS_2_is_arthritis | DGNS_3_is_arthritis | DGNS_4_is_arthritis | DGNS_6_is_arthritis | DGNS_7_is_arthritis | 
  DGNS_8_is_arthritis | DGNS_9_is_arthritis | DGNS_10_is_arthritis | DGNS_11_is_arthritis | DGNS_12_is_arthritis,

  
  is_IHD=
  PRNCPAL_DGNS_is_IHD | DGNS_1_is_IHD | DGNS_2_is_IHD | DGNS_3_is_IHD | DGNS_4_is_IHD | DGNS_6_is_IHD | DGNS_7_is_IHD | 
  DGNS_8_is_IHD | DGNS_9_is_IHD | DGNS_10_is_IHD | DGNS_11_is_IHD | DGNS_12_is_IHD,
  
  
  is_diabetes=
  PRNCPAL_DGNS_is_diabetes | DGNS_1_is_diabetes | DGNS_2_is_diabetes | DGNS_3_is_diabetes | DGNS_4_is_diabetes | DGNS_6_is_diabetes | DGNS_7_is_diabetes | 
  DGNS_8_is_diabetes | DGNS_9_is_diabetes | DGNS_10_is_diabetes | DGNS_11_is_diabetes | DGNS_12_is_diabetes,


  is_depression=
  PRNCPAL_DGNS_is_depression | DGNS_1_is_depression | DGNS_2_is_depression | DGNS_3_is_depression | DGNS_4_is_depression | DGNS_6_is_depression | DGNS_7_is_depression | 
  DGNS_8_is_depression | DGNS_9_is_depression | DGNS_10_is_depression | DGNS_11_is_depression | DGNS_12_is_depression
)%>%
as.data.table()

In [ ]:
head(conditions_claim_carrier)

DESY_SORT_KEY,year,date,month_year,is_hypertension,is_arthritis,is_IHD,is_diabetes,is_depression
<int>,<chr>,<date>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100000015,2013,2013-04-25,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2013,2013-05-28,2013-05,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,2013,2013-07-19,2013-07,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2013,2013-07-19,2013-07,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2013,2013-10-21,2013-10,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,2013,2013-11-05,2013-11,FALSE,FALSE,FALSE,FALSE,FALSE


### Inpatient

In [ ]:
conditions_claim_inpatient=
claim_inpatient_all_years %>%
summarise(
  
  DESY_SORT_KEY = DESY_SORT_KEY,
  
  year=year,
  
  date=date,
  
  month_year=month_year,
  
  PRNCPAL_DGNS_is_hypertension=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes)),
  PRNCPAL_DGNS_is_arthritis=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes)),
  PRNCPAL_DGNS_is_IHD=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% IHD_icd_9_codes)),
  PRNCPAL_DGNS_is_diabetes=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes)),
  PRNCPAL_DGNS_is_depression=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 4) %in% depression_icd_9_codes)),
  
  ADMTG_DGNS_is_hypertension=(
    case_when(
      ADMTG_DGNS_VRSN_CD == 0 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
      ADMTG_DGNS_VRSN_CD == 9 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes)),
  ADMTG_DGNS_is_arthritis=(
    case_when(
      ADMTG_DGNS_VRSN_CD == 0 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
      ADMTG_DGNS_VRSN_CD == 9 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes)),
  ADMTG_DGNS_is_IHD=(
    case_when(
      ADMTG_DGNS_VRSN_CD == 0 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
      ADMTG_DGNS_VRSN_CD == 9 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% IHD_icd_9_codes)),
  ADMTG_DGNS_is_diabetes=(
    case_when(
      ADMTG_DGNS_VRSN_CD == 0 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
      ADMTG_DGNS_VRSN_CD == 9 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes)),
  ADMTG_DGNS_is_depression=(
    case_when(
      ADMTG_DGNS_VRSN_CD == 0 ~ substr(ADMTG_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
      ADMTG_DGNS_VRSN_CD == 9 ~ substr(ADMTG_DGNS_CD, 0, 4) %in% depression_icd_9_codes)),
  
  
  DGNS_1_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_2_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% hypertension_icd_9_codes)),

  DGNS_3_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_4_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_6_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_7_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_8_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_9_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_10_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_11_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_12_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_13_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_14_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_15_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_16_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_17_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_18_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_19_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_20_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_21_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_22_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_23_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_24_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_25_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% hypertension_icd_9_codes)),
  
  

  DGNS_1_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_2_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% arthritis_icd_9_codes)),

  DGNS_3_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_4_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_6_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_7_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_8_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_9_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_10_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_11_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_12_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_13_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_14_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_15_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_16_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_17_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_18_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_19_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_20_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_21_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_22_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_23_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_24_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_25_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% arthritis_icd_9_codes)),






  DGNS_1_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_2_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% IHD_icd_9_codes)),

  DGNS_3_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_4_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_6_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_7_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_8_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_9_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_10_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_11_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_12_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_13_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_14_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_15_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_16_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_17_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_18_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_19_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_20_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_21_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_22_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_23_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_24_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_25_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% IHD_icd_9_codes)),


  DGNS_1_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_2_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% diabetes_icd_9_codes)),

  DGNS_3_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_4_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_6_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_7_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_8_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_9_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_10_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_11_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_12_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_13_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_14_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_15_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_16_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_17_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_18_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_19_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_20_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_21_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_22_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_23_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_24_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_25_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% diabetes_icd_9_codes)),


  DGNS_1_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_2_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 4) %in% depression_icd_9_codes)),

  DGNS_3_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_4_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_6_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_7_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_8_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_9_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_10_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_11_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_12_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_13_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_14_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_15_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_16_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_17_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_18_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_19_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_20_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_21_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_22_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_23_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_24_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_25_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 4) %in% depression_icd_9_codes)),


  
)%>%
as.data.table()

conditions_claim_inpatient[is.na(conditions_claim_inpatient)] <- 0

conditions_claim_inpatient=
conditions_claim_inpatient%>%
summarise(
  
  DESY_SORT_KEY = DESY_SORT_KEY,
  
  year=year,
  
  date=date,
  
  month_year=month_year,
  

  is_hypertension=
  ADMTG_DGNS_is_hypertension | PRNCPAL_DGNS_is_hypertension | DGNS_1_is_hypertension | DGNS_2_is_hypertension | DGNS_3_is_hypertension | DGNS_4_is_hypertension | DGNS_6_is_hypertension | DGNS_7_is_hypertension | 
  DGNS_8_is_hypertension | DGNS_9_is_hypertension | DGNS_10_is_hypertension | DGNS_11_is_hypertension | DGNS_12_is_hypertension| DGNS_13_is_hypertension | DGNS_14_is_hypertension | DGNS_15_is_hypertension |
  DGNS_16_is_hypertension | DGNS_17_is_hypertension | DGNS_18_is_hypertension | DGNS_19_is_hypertension | DGNS_20_is_hypertension | DGNS_21_is_hypertension | DGNS_22_is_hypertension | DGNS_23_is_hypertension|
  DGNS_24_is_hypertension | DGNS_25_is_hypertension,
  
  is_arthritis=
  ADMTG_DGNS_is_arthritis | PRNCPAL_DGNS_is_arthritis | DGNS_1_is_arthritis | DGNS_2_is_arthritis | DGNS_3_is_arthritis | DGNS_4_is_arthritis | DGNS_6_is_arthritis | DGNS_7_is_arthritis | 
  DGNS_8_is_arthritis | DGNS_9_is_arthritis | DGNS_10_is_arthritis | DGNS_11_is_arthritis | DGNS_12_is_arthritis| DGNS_13_is_arthritis | DGNS_14_is_arthritis | DGNS_15_is_arthritis |
  DGNS_16_is_arthritis | DGNS_17_is_arthritis | DGNS_18_is_arthritis | DGNS_19_is_arthritis | DGNS_20_is_arthritis | DGNS_21_is_arthritis | DGNS_22_is_arthritis | DGNS_23_is_arthritis|
  DGNS_24_is_arthritis | DGNS_25_is_arthritis,
  

  
  is_IHD=
  ADMTG_DGNS_is_IHD | PRNCPAL_DGNS_is_IHD | DGNS_1_is_IHD | DGNS_2_is_IHD | DGNS_3_is_IHD | DGNS_4_is_IHD | DGNS_6_is_IHD | DGNS_7_is_IHD | 
  DGNS_8_is_IHD | DGNS_9_is_IHD | DGNS_10_is_IHD | DGNS_11_is_IHD | DGNS_12_is_IHD| DGNS_13_is_IHD | DGNS_14_is_IHD | DGNS_15_is_IHD |
  DGNS_16_is_IHD | DGNS_17_is_IHD | DGNS_18_is_IHD | DGNS_19_is_IHD | DGNS_20_is_IHD | DGNS_21_is_IHD | DGNS_22_is_IHD | DGNS_23_is_IHD|
  DGNS_24_is_IHD | DGNS_25_is_IHD,
  
  
  
  is_diabetes=
  ADMTG_DGNS_is_diabetes | PRNCPAL_DGNS_is_diabetes | DGNS_1_is_diabetes | DGNS_2_is_diabetes | DGNS_3_is_diabetes | DGNS_4_is_diabetes | DGNS_6_is_diabetes | DGNS_7_is_diabetes | 
  DGNS_8_is_diabetes | DGNS_9_is_diabetes | DGNS_10_is_diabetes | DGNS_11_is_diabetes | DGNS_12_is_diabetes| DGNS_13_is_diabetes | DGNS_14_is_diabetes | DGNS_15_is_diabetes |
  DGNS_16_is_diabetes | DGNS_17_is_diabetes | DGNS_18_is_diabetes | DGNS_19_is_diabetes | DGNS_20_is_diabetes | DGNS_21_is_diabetes | DGNS_22_is_diabetes | DGNS_23_is_diabetes|
  DGNS_24_is_diabetes | DGNS_25_is_diabetes,
  


  is_depression=
  ADMTG_DGNS_is_depression | PRNCPAL_DGNS_is_depression | DGNS_1_is_depression | DGNS_2_is_depression | DGNS_3_is_depression | DGNS_4_is_depression | DGNS_6_is_depression | DGNS_7_is_depression | 
  DGNS_8_is_depression | DGNS_9_is_depression | DGNS_10_is_depression | DGNS_11_is_depression | DGNS_12_is_depression| DGNS_13_is_depression | DGNS_14_is_depression | DGNS_15_is_depression |
  DGNS_16_is_depression | DGNS_17_is_depression | DGNS_18_is_depression | DGNS_19_is_depression | DGNS_20_is_depression | DGNS_21_is_depression | DGNS_22_is_depression | DGNS_23_is_depression|
  DGNS_24_is_depression | DGNS_25_is_depression,
  
)%>%
as.data.table()

In [ ]:
tail(conditions_claim_inpatient)

DESY_SORT_KEY,year,date,month_year,is_hypertension,is_arthritis,is_IHD,is_diabetes,is_depression
<int>,<chr>,<date>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100014409,2013,2013-01-25,2013-01,FALSE,FALSE,TRUE,TRUE,FALSE
100014493,2013,2013-07-15,2013-07,TRUE,FALSE,FALSE,FALSE,FALSE
100014605,2013,2013-10-20,2013-10,TRUE,FALSE,FALSE,FALSE,FALSE
100014605,2013,2013-10-31,2013-10,TRUE,FALSE,FALSE,FALSE,FALSE
100015173,2013,2013-09-26,2013-09,TRUE,FALSE,FALSE,FALSE,FALSE
100015307,2013,2013-08-13,2013-08,FALSE,FALSE,FALSE,FALSE,FALSE


### Outpatient

In [ ]:
conditions_claim_outpatient=
claim_outpatient_all_years %>%
summarise(
  
  DESY_SORT_KEY = DESY_SORT_KEY,
  
  year=year,
  
  date=date,
  
  month_year=month_year,
  
  PRNCPAL_DGNS_is_hypertension=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% hypertension_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% hypertension_icd_9_codes)),
  PRNCPAL_DGNS_is_arthritis=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% arthritis_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% arthritis_icd_9_codes)),
  PRNCPAL_DGNS_is_IHD=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% IHD_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% IHD_icd_9_codes)),
  PRNCPAL_DGNS_is_diabetes=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% diabetes_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% diabetes_icd_9_codes)),
  PRNCPAL_DGNS_is_depression=(
    case_when(
      PRNCPAL_DGNS_VRSN_CD == 0 ~ substr(PRNCPAL_DGNS_CD, 0, 3) %in% depression_icd_10_codes,
      PRNCPAL_DGNS_VRSN_CD == 9 ~ substr(PRNCPAL_DGNS_CD, 0, 4) %in% depression_icd_9_codes)),
  
  
  DGNS_1_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_2_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% hypertension_icd_9_codes)),

  DGNS_3_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_4_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_6_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_7_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_8_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_9_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_10_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_11_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_12_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_13_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_14_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_15_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_16_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_17_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_18_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_19_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_20_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_21_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_22_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_23_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_24_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% hypertension_icd_9_codes)),
  
  DGNS_25_is_hypertension=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% hypertension_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% hypertension_icd_9_codes)),
  
  

  DGNS_1_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_2_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% arthritis_icd_9_codes)),

  DGNS_3_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_4_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_6_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_7_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_8_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_9_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_10_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_11_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_12_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_13_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_14_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_15_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_16_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_17_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_18_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_19_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_20_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_21_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_22_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_23_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_24_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% arthritis_icd_9_codes)),
  
  DGNS_25_is_arthritis=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% arthritis_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% arthritis_icd_9_codes)),






  DGNS_1_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_2_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% IHD_icd_9_codes)),

  DGNS_3_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_4_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_6_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_7_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_8_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_9_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_10_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_11_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_12_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_13_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_14_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_15_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_16_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_17_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_18_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_19_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_20_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_21_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_22_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_23_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_24_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% IHD_icd_9_codes)),
  
  DGNS_25_is_IHD=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% IHD_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% IHD_icd_9_codes)),


  DGNS_1_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_2_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 3) %in% diabetes_icd_9_codes)),

  DGNS_3_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_4_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_6_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_7_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_8_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_9_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_10_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_11_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_12_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_13_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_14_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_15_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_16_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_17_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_18_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_19_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_20_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_21_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_22_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_23_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_24_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 3) %in% diabetes_icd_9_codes)),
  
  DGNS_25_is_diabetes=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% diabetes_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 3) %in% diabetes_icd_9_codes)),


  DGNS_1_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD1 == 0 ~ substr(ICD_DGNS_CD1, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD1 == 9 ~ substr(ICD_DGNS_CD1, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_2_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD2 == 0 ~ substr(ICD_DGNS_CD2, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD2 == 9 ~ substr(ICD_DGNS_CD2, 0, 4) %in% depression_icd_9_codes)),

  DGNS_3_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD3 == 0 ~ substr(ICD_DGNS_CD3, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD3 == 9 ~ substr(ICD_DGNS_CD3, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_4_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD4 == 0 ~ substr(ICD_DGNS_CD4, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD4 == 9 ~ substr(ICD_DGNS_CD4, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_6_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD6 == 0 ~ substr(ICD_DGNS_CD6, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD6 == 9 ~ substr(ICD_DGNS_CD6, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_7_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD7 == 0 ~ substr(ICD_DGNS_CD7, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD7 == 9 ~ substr(ICD_DGNS_CD7, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_8_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD8 == 0 ~ substr(ICD_DGNS_CD8, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD8 == 9 ~ substr(ICD_DGNS_CD8, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_9_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD9 == 0 ~ substr(ICD_DGNS_CD9, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD9 == 9 ~ substr(ICD_DGNS_CD9, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_10_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD10 == 0 ~ substr(ICD_DGNS_CD10, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD10 == 9 ~ substr(ICD_DGNS_CD10, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_11_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD11 == 0 ~ substr(ICD_DGNS_CD11, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD11 == 9 ~ substr(ICD_DGNS_CD11, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_12_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD12 == 0 ~ substr(ICD_DGNS_CD12, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD12 == 9 ~ substr(ICD_DGNS_CD12, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_13_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD13 == 0 ~ substr(ICD_DGNS_CD13, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD13 == 9 ~ substr(ICD_DGNS_CD13, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_14_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD14 == 0 ~ substr(ICD_DGNS_CD14, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD14 == 9 ~ substr(ICD_DGNS_CD14, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_15_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD15 == 0 ~ substr(ICD_DGNS_CD15, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD15 == 9 ~ substr(ICD_DGNS_CD15, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_16_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD16 == 0 ~ substr(ICD_DGNS_CD16, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD16 == 9 ~ substr(ICD_DGNS_CD16, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_17_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD17 == 0 ~ substr(ICD_DGNS_CD17, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD17 == 9 ~ substr(ICD_DGNS_CD17, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_18_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD18 == 0 ~ substr(ICD_DGNS_CD18, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD18 == 9 ~ substr(ICD_DGNS_CD18, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_19_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD19 == 0 ~ substr(ICD_DGNS_CD19, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD19 == 9 ~ substr(ICD_DGNS_CD19, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_20_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD20 == 0 ~ substr(ICD_DGNS_CD20, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD20 == 9 ~ substr(ICD_DGNS_CD20, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_21_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD21 == 0 ~ substr(ICD_DGNS_CD21, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD21 == 9 ~ substr(ICD_DGNS_CD21, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_22_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD22 == 0 ~ substr(ICD_DGNS_CD22, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD22 == 9 ~ substr(ICD_DGNS_CD22, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_23_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD23 == 0 ~ substr(ICD_DGNS_CD23, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD23 == 9 ~ substr(ICD_DGNS_CD23, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_24_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD24 == 0 ~ substr(ICD_DGNS_CD24, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD24 == 9 ~ substr(ICD_DGNS_CD24, 0, 4) %in% depression_icd_9_codes)),
  
  DGNS_25_is_depression=(
    case_when(
      ICD_DGNS_VRSN_CD25 == 0 ~ substr(ICD_DGNS_CD25, 0, 3) %in% depression_icd_10_codes,
      ICD_DGNS_VRSN_CD25 == 9 ~ substr(ICD_DGNS_CD25, 0, 4) %in% depression_icd_9_codes)),


  
)%>%
as.data.table()

conditions_claim_outpatient[is.na(conditions_claim_outpatient)] <- 0

conditions_claim_outpatient=
conditions_claim_outpatient%>%
summarise(
  
  DESY_SORT_KEY = DESY_SORT_KEY,
  
  year=year,
  
  date=date,
  
  month_year=month_year,
  

  is_hypertension=
 PRNCPAL_DGNS_is_hypertension | DGNS_1_is_hypertension | DGNS_2_is_hypertension | DGNS_3_is_hypertension | DGNS_4_is_hypertension | DGNS_6_is_hypertension | DGNS_7_is_hypertension | 
  DGNS_8_is_hypertension | DGNS_9_is_hypertension | DGNS_10_is_hypertension | DGNS_11_is_hypertension | DGNS_12_is_hypertension| DGNS_13_is_hypertension | DGNS_14_is_hypertension | DGNS_15_is_hypertension |
  DGNS_16_is_hypertension | DGNS_17_is_hypertension | DGNS_18_is_hypertension | DGNS_19_is_hypertension | DGNS_20_is_hypertension | DGNS_21_is_hypertension | DGNS_22_is_hypertension | DGNS_23_is_hypertension|
  DGNS_24_is_hypertension | DGNS_25_is_hypertension,
  
  is_arthritis=
 PRNCPAL_DGNS_is_arthritis | DGNS_1_is_arthritis | DGNS_2_is_arthritis | DGNS_3_is_arthritis | DGNS_4_is_arthritis | DGNS_6_is_arthritis | DGNS_7_is_arthritis | 
  DGNS_8_is_arthritis | DGNS_9_is_arthritis | DGNS_10_is_arthritis | DGNS_11_is_arthritis | DGNS_12_is_arthritis| DGNS_13_is_arthritis | DGNS_14_is_arthritis | DGNS_15_is_arthritis |
  DGNS_16_is_arthritis | DGNS_17_is_arthritis | DGNS_18_is_arthritis | DGNS_19_is_arthritis | DGNS_20_is_arthritis | DGNS_21_is_arthritis | DGNS_22_is_arthritis | DGNS_23_is_arthritis|
  DGNS_24_is_arthritis | DGNS_25_is_arthritis,
  

  
  is_IHD=
 PRNCPAL_DGNS_is_IHD | DGNS_1_is_IHD | DGNS_2_is_IHD | DGNS_3_is_IHD | DGNS_4_is_IHD | DGNS_6_is_IHD | DGNS_7_is_IHD | 
  DGNS_8_is_IHD | DGNS_9_is_IHD | DGNS_10_is_IHD | DGNS_11_is_IHD | DGNS_12_is_IHD| DGNS_13_is_IHD | DGNS_14_is_IHD | DGNS_15_is_IHD |
  DGNS_16_is_IHD | DGNS_17_is_IHD | DGNS_18_is_IHD | DGNS_19_is_IHD | DGNS_20_is_IHD | DGNS_21_is_IHD | DGNS_22_is_IHD | DGNS_23_is_IHD|
  DGNS_24_is_IHD | DGNS_25_is_IHD,
  
  
  
  is_diabetes=
 PRNCPAL_DGNS_is_diabetes | DGNS_1_is_diabetes | DGNS_2_is_diabetes | DGNS_3_is_diabetes | DGNS_4_is_diabetes | DGNS_6_is_diabetes | DGNS_7_is_diabetes | 
  DGNS_8_is_diabetes | DGNS_9_is_diabetes | DGNS_10_is_diabetes | DGNS_11_is_diabetes | DGNS_12_is_diabetes| DGNS_13_is_diabetes | DGNS_14_is_diabetes | DGNS_15_is_diabetes |
  DGNS_16_is_diabetes | DGNS_17_is_diabetes | DGNS_18_is_diabetes | DGNS_19_is_diabetes | DGNS_20_is_diabetes | DGNS_21_is_diabetes | DGNS_22_is_diabetes | DGNS_23_is_diabetes|
  DGNS_24_is_diabetes | DGNS_25_is_diabetes,
  


  is_depression=
 PRNCPAL_DGNS_is_depression | DGNS_1_is_depression | DGNS_2_is_depression | DGNS_3_is_depression | DGNS_4_is_depression | DGNS_6_is_depression | DGNS_7_is_depression | 
  DGNS_8_is_depression | DGNS_9_is_depression | DGNS_10_is_depression | DGNS_11_is_depression | DGNS_12_is_depression| DGNS_13_is_depression | DGNS_14_is_depression | DGNS_15_is_depression |
  DGNS_16_is_depression | DGNS_17_is_depression | DGNS_18_is_depression | DGNS_19_is_depression | DGNS_20_is_depression | DGNS_21_is_depression | DGNS_22_is_depression | DGNS_23_is_depression|
  DGNS_24_is_depression | DGNS_25_is_depression,
  
)%>%
as.data.table()

In [ ]:
head(conditions_claim_outpatient)

DESY_SORT_KEY,year,date,month_year,is_hypertension,is_arthritis,is_IHD,is_diabetes,is_depression
<int>,<chr>,<date>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100000015,2013,2013-04-25,2013-04,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2013,2013-07-19,2013-07,FALSE,FALSE,FALSE,FALSE,FALSE
100000015,2013,2013-10-14,2013-10,FALSE,FALSE,FALSE,FALSE,FALSE
100000019,2013,2013-05-23,2013-05,TRUE,FALSE,FALSE,FALSE,FALSE
100000019,2013,2013-05-29,2013-05,FALSE,FALSE,FALSE,FALSE,FALSE
100000019,2013,2013-06-03,2013-06,FALSE,FALSE,FALSE,FALSE,FALSE


### Summarizing claim level patient data

In [ ]:
claim_level_all_data=rbind(conditions_claim_carrier,conditions_claim_inpatient)
claim_level_all_data=rbind(claim_level_all_data,conditions_claim_outpatient)

In [ ]:
summarise_claim_level <- function(data, time_frame = 365) {
  data %>%
    group_by(DESY_SORT_KEY, year) %>%
    summarise(
      hypertension = sum(is_hypertension, na.rm = T) > 0,
      arthritis = sum(is_arthritis, na.rm = T) > 0,
      IHD = sum(is_IHD, na.rm = T) > 0,
      diabetes = sum(is_diabetes, na.rm = T) > 0,
      depression = sum(is_depression, na.rm = T) > 0,
    ) %>%
    as.data.table()
}

summary_claim_level_patient_by_year <- summarise_claim_level(claim_level_all_data)

`summarise()` has grouped output by 'DESY_SORT_KEY'. You can override using the `.groups` argument.


In [ ]:
tail(summary_claim_level_patient_by_year)

DESY_SORT_KEY,year,hypertension,arthritis,IHD,diabetes,depression
<int>,<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100014285,2013,TRUE,FALSE,FALSE,FALSE,FALSE
100014409,2013,TRUE,FALSE,TRUE,TRUE,FALSE
100014493,2013,TRUE,FALSE,FALSE,FALSE,FALSE
100014605,2013,TRUE,FALSE,FALSE,FALSE,FALSE
100015173,2013,TRUE,FALSE,FALSE,FALSE,FALSE
100015307,2013,FALSE,FALSE,FALSE,FALSE,FALSE


In [ ]:
write_fst(summary_claim_level_patient_by_year, "/work/postresearch/Shared/Projects/Farbod/CaseMix/summary_claim_level_patient_by_year.fst")

## Reading the claim level conditions results

In [ ]:
summary_claim_level_patient_by_year=read_fst("/work/postresearch/Shared/Projects/Farbod/CaseMix/summary_claim_level_patient_by_year.fst",as.data.table = T)

In [11]:
summary_claim_level_patient_by_year[,year:=as.double(year)]

In [12]:
head(summary_claim_level_patient_by_year)

DESY_SORT_KEY,year,hypertension,arthritis,IHD,diabetes,depression
<int>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
100000015,2013,FALSE,FALSE,TRUE,FALSE,FALSE
100000015,2014,TRUE,FALSE,TRUE,TRUE,FALSE
100000015,2015,TRUE,FALSE,TRUE,TRUE,FALSE
100000015,2016,TRUE,FALSE,TRUE,TRUE,FALSE
100000015,2017,TRUE,FALSE,TRUE,TRUE,FALSE
100000015,2018,TRUE,FALSE,TRUE,TRUE,FALSE


## Adding to the main data

In [ ]:
yearly_calculations <- read_fst("yearly_calculations_with_integration_changes_mdppas.fst", as.data.table = T)

In [6]:
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<d

In [30]:
yearly_calculations=
yearly_calculations %>%
left_join(.,summary_claim_level_patient_by_year,by=c("DESY_SORT_KEY","year"),suffix = c("_old","_carrier"))%>%
mutate(hypertension=hypertension_old|hypertension_carrier,
       arthritis=arthritis_old|arthritis_carrier,
       IHD=IHD_old|IHD_carrier,
       diabetes=diabetes_old|diabetes_carrier,
       depression=depression_old|depression_carrier
      )%>%
as.data.table()

In [31]:
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension_old,arthritis_old,IHD_old,diabetes_old,depression_old,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration,hypertension_carrier,arthritis_carrier,IHD_carrier,diabetes_carrier,depression_carrier,hypertension,arthritis,IHD,diabetes,depression
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr

## Counting number of conditions

In [32]:
yearly_calculations[, number_of_conditions := sum(hypertension, arthritis, IHD, diabetes, depression, na.rm = T), ,
  by = 1:nrow(yearly_calculations)
]

In [33]:
head(yearly_calculations)

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension_old,arthritis_old,IHD_old,diabetes_old,depression_old,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration,hypertension_carrier,arthritis_carrier,IHD_carrier,diabetes_carrier,depression_carrier,hypertension,arthritis,IHD,diabetes,depression
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr

In [34]:
write_fst(yearly_calculations, "yearly_calculations_with_integration_changes_mdppas_claim_level.fst")

## Patient pool changes

### Read cleaned results

In [5]:
# yearly_calculations <- read_fst("yearly_calculations_cleaned.fst", as.data.table = T)
yearly_calculations <- read_fst("yearly_calculations_with_integration_changes_mdppas_claim_level.fst", as.data.table = T)

In [35]:
yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension_old,arthritis_old,IHD_old,diabetes_old,depression_old,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration,hypertension_carrier,arthritis_carrier,IHD_carrier,diabetes_carrier,depression_carrier,hypertension,arthritis,IHD,diabetes,depression
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr

### Summarize Changes

In [36]:
integration_change_summarizer <- function(data) {
  require(tidyverse)
  require(dtplyr)

  result <-
    data %>%
    filter(!is.na(most_common_primary_care_physician_years_from_integration)) %>%
    group_by(most_common_primary_care_physician_years_from_integration) %>%
    summarise(
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T)
    ) %>%
    as.data.table()
}

In [37]:
integration_changes_years_to_integration <- integration_change_summarizer(yearly_calculations)

In [39]:
integration_changes_years_to_integration

most_common_primary_care_physician_years_from_integration,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-7,0.8102548,0.03715075,0.3196193,0.3131716,0.02272029,0.3423396,0.1449186
-6,0.8042567,0.04043041,0.2989161,0.3238389,0.02587230,0.3331751,0.1474009
-5,0.8019594,0.04343364,0.3007231,0.3277817,0.03648239,0.3254957,0.1580126
-4,0.8001693,0.05397184,0.2895537,0.3203500,0.05591620,0.3245210,0.1604102
-3,0.7936568,0.07110350,0.2871118,0.3247865,0.07734353,0.3230037,0.1734541
-2,0.7934929,0.07958879,0.2849168,0.3220214,0.08710551,0.3229242,0.1779510
-1,0.7949667,0.08310343,0.2824844,0.3196286,0.09560258,0.3208566,0.1813472
0,0.7882874,0.09550270,0.2762273,0.3143743,0.11891532,0.3163306,0.1900273
1,0.7830947,0.10453509,0.2779487,0.3087581,0.14278126,0.3132511,0.1996414


In [40]:
ggplot_all_variables <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_all_variables, filename = "ggplot_all_variables.pdf", width = 12, height = 7)

In [47]:
ggplot_hypertension <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_hypertension, filename = "ggplot_hypertension.pdf", width = 12, height = 7)

In [41]:
ggplot_arthritis_depression_more_than_two <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_arthritis_depression_more_than_two, filename = "ggplot_arthritis_depression_more_than_two.pdf", width = 12, height = 7)

In [42]:
ggplot_IHD_diabetes_two_conditions <- ggplot(data = integration_changes_years_to_integration) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence")

ggsave(ggplot_IHD_diabetes_two_conditions, filename = "ggplot_IHD_diabetes_two_conditions.pdf", width = 12, height = 7)

In [43]:
ggplot_hypertension_rdd <- ggplot(data = integration_changes_years_to_integration %>% mutate(before = (most_common_primary_care_physician_years_from_integration < 0)) %>% as.data.frame()) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence") +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = hypertension, color = "hypertension", fill = "hypertension", group = before))



ggsave(ggplot_hypertension_rdd, filename = "ggplot_hypertension_rdd.pdf", width = 12, height = 7)

`geom_smooth()` using formula 'y ~ x'



In [44]:
ggplot_IHD_diabetes_two_conditions_rdd <- ggplot(data = integration_changes_years_to_integration %>% mutate(before = (most_common_primary_care_physician_years_from_integration < 0)) %>% as.data.frame()) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence") +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = IHD, color = "IHD", fill = "IHD", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = diabetes, color = "diabetes", fill = "diabetes", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = two_conditions, color = "two_conditions", fill = "two_conditions", group = before))



ggsave(ggplot_IHD_diabetes_two_conditions_rdd, filename = "ggplot_IHD_diabetes_two_conditions_rdd.pdf", width = 12, height = 7)

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



In [45]:
ggplot_arthritis_depression_more_than_two_rdd <-
  ggplot(data = integration_changes_years_to_integration %>% mutate(before = (most_common_primary_care_physician_years_from_integration < 0)) %>% as.data.frame()) +
  geom_vline(xintercept = 0, linetype = "dotted", color = "black", size = 1.5) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression")) +
  geom_line(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  geom_point(aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions")) +
  scale_colour_discrete() +
  scale_x_continuous(breaks = seq(-8, 8, 2)) +
  xlab("Primary care physician years from integration") +
  ylab("Prevalence") +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = arthritis, color = "arthritis", fill = "arthritis", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = depression, color = "depression", fill = "depression", group = before)) +
  geom_smooth(method = "lm", aes(x = most_common_primary_care_physician_years_from_integration, y = more_than_two_conditions, color = "more_than_two_conditions", fill = "more_than_two_conditions", group = before))

ggsave(ggplot_arthritis_depression_more_than_two_rdd, filename = "ggplot_arthritis_depression_more_than_two_rdd.pdf", width = 12, height = 7)

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'

`geom_smooth()` using formula 'y ~ x'



# Comparisons based on integration status

## Read cleaned results

In [3]:
# yearly_calculations <- read_fst("yearly_calculations_cleaned.fst", as.data.table = T)
yearly_calculations <- read_fst("yearly_calculations_with_integration_changes_mdppas_claim_level.fst", as.data.table = T)

In [4]:
yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>,<int>,<lgl>,<dbl>,<dbl>,<lgl>
100005325,2013,10,3,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,13,20,140,2,1,76,0,0,C,C,3,3,3,3,3,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0

## Comparison functions

In [48]:
comparator_averages <- function(data) {
  require(tidyverse)
  require(dtplyr)


  result <- data %>%
    group_by(most_common_primary_care_physician_integrated) %>%
    summarise(
      n = n(),
      N_distinct_patients = length(unique(DESY_SORT_KEY)),
      N_distinct_specialist = length(unique(most_common_primary_care_physician_PRF_PHYSN_NPI)),
      Male = mean(sex == "Male", na.rm = T),
      Female = mean(sex == "Female", na.rm = T),
      Asian = mean(race == "Asian", na.rm = T),
      Black = mean(race == "Black", na.rm = T),
      Hispanic = mean(race == "Hispanic", na.rm = T),
      North_American_Native = mean(race == "North American Native", na.rm = T),
      White = mean(race == "White", na.rm = T),
      Other_race = mean(race == "Other", na.rm = T),
      avg_age = mean(AGE, na.rm = T),
      age_65_74 = mean(na.rm = T, age_group == "65-74"),
      age_75_84 = mean(na.rm = T, age_group == "75-84"),
      age_85_plus = mean(na.rm = T, age_group == "85+"),

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = mean(RUCC_2013 > 3, na.rm = T),
      urban_prp = mean(RUCC_2013 <= 3, na.rm = T),
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T),
      # primary_care_physician_integrated =
      #  sum(most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n(),
      # most_common_physician_integrated =
      #  sum(most_common_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n()
    ) %>%
    as.data.table()

  return(result)
}

In [49]:
comparator_tests <- function(data) {
  require(tidyverse)
  require(dtplyr)

  result <- data %>%
    mutate(
      integration_status = most_common_primary_care_physician_integrated
    ) %>%
    summarise(
      Male = chisq.test(sex == "Male", integration_status)$p.value,
      Female = chisq.test(sex == "Female", integration_status)$p.value,
      Asian = chisq.test(race == "Asian", integration_status)$p.value,
      Black = chisq.test(race == "Black", integration_status)$p.value,
      Hispanic = chisq.test(race == "Hispanic", integration_status)$p.value,
      North_American_Native = chisq.test(race == "North American Native", integration_status)$p.value,
      White = chisq.test(race == "White", integration_status)$p.value,
      Other_race = chisq.test(race == "Other", integration_status)$p.value,
      avg_age = t.test(AGE ~ integration_status, data = .)$p.value,
      age_65_74 = chisq.test(age_group == "65-74", integration_status)$p.value,
      age_75_84 = chisq.test(age_group == "75-84", integration_status)$p.value,
      age_85_plus = chisq.test(age_group == "85+", integration_status)$p.value,

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = chisq.test(RUCC_2013 > 3, integration_status)$p.value,
      urban_prp = chisq.test(RUCC_2013 <= 3, integration_status)$p.value,
      hypertension = chisq.test(hypertension, integration_status)$p.value,
      arthritis = chisq.test(arthritis, integration_status)$p.value,
      IHD = chisq.test(IHD, integration_status)$p.value,
      diabetes = chisq.test(diabetes, integration_status)$p.value,
      depression = chisq.test(depression, integration_status)$p.value,
      two_conditions = chisq.test(number_of_conditions == 2, integration_status)$p.value,
      more_than_two_conditions = chisq.test(number_of_conditions > 2, integration_status)$p.value,
    ) %>%
    as.data.table()

  return(result)
}

In [50]:
overall_averages <- function(data) {
  require(tidyverse)
  require(dtplyr)


  result <- data %>%
    summarise(
      n = n(),
      N_distinct_patients = length(unique(DESY_SORT_KEY)),
      N_distinct_specialist = length(unique(most_common_primary_care_physician_PRF_PHYSN_NPI)),
      Male = mean(sex == "Male", na.rm = T),
      Female = mean(sex == "Female", na.rm = T),
      Asian = mean(race == "Asian", na.rm = T),
      Black = mean(race == "Black", na.rm = T),
      Hispanic = mean(race == "Hispanic", na.rm = T),
      North_American_Native = mean(race == "North American Native", na.rm = T),
      White = mean(race == "White", na.rm = T),
      Other_race = mean(race == "Other", na.rm = T),
      avg_age = mean(AGE, na.rm = T),
      age_65_74 = mean(na.rm = T, age_group == "65-74"),
      age_75_84 = mean(na.rm = T, age_group == "75-84"),
      age_85_plus = mean(na.rm = T, age_group == "85+"),

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = mean(RUCC_2013 > 3, na.rm = T),
      urban_prp = mean(RUCC_2013 <= 3, na.rm = T),
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T),
      # primary_care_physician_integrated =
      #  sum(most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n(),
      # most_common_physician_integrated =
      #  sum(most_common_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n()
    ) %>%
    as.data.table()

  return(result)
}

In [51]:
primary_care_physician_comparisons_averages <-
  comparator_averages(yearly_calculations)

primary_care_physician_comparisons_test <-
  comparator_tests(yearly_calculations)

primary_care_physician_overall_averages <-
  overall_averages(yearly_calculations)

In [52]:
primary_care_physician_comparisons_averages

most_common_primary_care_physician_integrated,n,N_distinct_patients,N_distinct_specialist,Male,Female,Asian,Black,Hispanic,North_American_Native,White,Other_race,avg_age,age_65_74,age_75_84,age_85_plus,rural_prp,urban_prp,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions
<lgl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,3508367,1662616,110213,0.4403838,0.5596162,0.01597723,0.07343987,0.01588061,0.005061614,0.8584598,0.01459169,75.39474,0.5427197,0.2915516,0.1657287,0.2970823,0.7029177,0.6387162,0.06896570,0.2253647,0.2583156,0.1159956,0.2569999,0.1492885
FALSE,5308543,1419643,140805,0.4206246,0.5793754,0.02000191,0.06666311,0.01232994,0.002514814,0.8686523,0.01671739,75.12492,0.5350116,0.3283278,0.1366607,0.1752896,0.8247104,0.7948656,0.09105907,0.2802697,0.3263242,0.1023104,0.3191399,0.1884798
TRUE,1195873,421753,59511,0.4198841,0.5801159,0.01343119,0.08168175,0.01094681,0.011753756,0.8543474,0.01379160,75.03840,0.5406444,0.3246331,0.1347225,0.2356522,0.7643478,0.7750079,0.08798426,0.2679415,0.3063486,0.1191197,0.3095897,0.1804163


In [53]:
primary_care_physician_comparisons_test

Male,Female,Asian,Black,Hispanic,North_American_Native,White,Other_race,avg_age,age_65_74,age_75_84,age_85_plus,rural_prp,urban_prp,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.1385695,0.1385695,0,0,6.946583e-36,0,0,4.026462e-116,1.044855e-29,6.573711e-29,7.509705e-15,2.417305e-08,0,0,0,3.435206e-26,9.979561e-163,0,0,2.411174e-91,9.155554e-93


In [54]:
primary_care_physician_overall_averages

n,N_distinct_patients,N_distinct_specialist,Male,Female,Asian,Black,Hispanic,North_American_Native,White,Other_race,avg_age,age_65_74,age_75_84,age_85_plus,rural_prp,urban_prp,hypertension,arthritis,IHD,diabetes,depression,two_conditions,more_than_two_conditions
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10012783,2109378,191459,0.4274596,0.5725404,0.01780694,0.07083136,0.01340886,0.004510634,0.8633725,0.01562313,75.20913,0.5383852,0.3150005,0.1466143,0.2256704,0.7743296,0.7377809,0.08295056,0.2595592,0.300109,0.1091131,0.2962261,0.1737846


In [55]:
comparison_results_primary_care_physician <- rbind(
  primary_care_physician_overall_averages,
  primary_care_physician_comparisons_averages,
  primary_care_physician_comparisons_test,
  fill = TRUE
)
t(comparison_results_primary_care_physician)

n,1.001278e+07,3.508367e+06,5.308543e+06,1.195873e+06,NA
N_distinct_patients,2.109378e+06,1.662616e+06,1.419643e+06,4.217530e+05,NA
N_distinct_specialist,1.914590e+05,1.102130e+05,1.408050e+05,5.951100e+04,NA
Male,4.274596e-01,4.403838e-01,4.206246e-01,4.198841e-01,1.385695e-01
Female,5.725404e-01,5.596162e-01,5.793754e-01,5.801159e-01,1.385695e-01
Asian,1.780694e-02,1.597723e-02,2.000191e-02,1.343119e-02,0.000000e+00
Black,7.083136e-02,7.343987e-02,6.666311e-02,8.168175e-02,0.000000e+00
Hispanic,1.340886e-02,1.588061e-02,1.232994e-02,1.094681e-02,6.946583e-36
North_American_Native,4.510634e-03,5.061614e-03,2.514814e-03,1.175376e-02,0.000000e+00
White,8.633725e-01,8.584598e-01,8.686523e-01,8.543474e-01,0.000000e+00
Other_race,1.562313e-02,1.459169e-02,1.671739e-02,1.379160e-02,4.026462e-116


# Finding the prevelances of the chronic conditions in patients entering and exiting

## Read cleaned results

In [7]:
# yearly_calculations <- read_fst("yearly_calculations_cleaned.fst", as.data.table = T)
yearly_calculations <- read_fst("yearly_calculations_with_integration_changes_mdppas_claim_level.fst", as.data.table = T)

In [8]:
yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension,arthritis,IHD,diabetes,depression,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<int>,<d

## Find entering, exiting, and remaining patients

In [56]:
yearly_calculations <- yearly_calculations %>%
  mutate(
    exit_enter_status = case_when(
      most_common_primary_care_physician_integration_change == "newly_integrated" &
        most_common_primary_care_physician_changed == "FALSE" ~ "Remaining",
      most_common_primary_care_physician_integration_change == "newly_integrated" &
        most_common_primary_care_physician_changed == "TRUE" ~ "Entering",
      last_year_most_common_primary_care_physician_integration_change == "newly_integrated" &
        most_common_primary_care_physician_changed == "TRUE" ~ "Exiting"
    )
  ) %>%
  as.data.table()
yearly_calculations[DESY_SORT_KEY == "100005325"]

DESY_SORT_KEY,year,distinct_clinicians,distinct_primary_care_physicians,hypertension_old,arthritis_old,IHD_old,diabetes_old,depression_old,icd_9_pure,icd_10_pure,REFERENCE_YEAR,STATE_CODE,COUNTY_CODE,SEX_CODE,RACE_CODE,AGE,ORIG_REASON_FOR_ENTITLEMENT,CURR_REASON_FOR_ENTITLEMENT,ENTITLEMENT_BUY_IN_IND01,ENTITLEMENT_BUY_IN_IND02,ENTITLEMENT_BUY_IN_IND03,ENTITLEMENT_BUY_IN_IND04,ENTITLEMENT_BUY_IN_IND05,ENTITLEMENT_BUY_IN_IND06,ENTITLEMENT_BUY_IN_IND07,ENTITLEMENT_BUY_IN_IND08,ENTITLEMENT_BUY_IN_IND09,ENTITLEMENT_BUY_IN_IND10,ENTITLEMENT_BUY_IN_IND11,ENTITLEMENT_BUY_IN_IND12,HMO_INDICATOR01,HMO_INDICATOR02,HMO_INDICATOR03,HMO_INDICATOR04,HMO_INDICATOR05,HMO_INDICATOR06,HMO_INDICATOR07,HMO_INDICATOR08,HMO_INDICATOR09,HMO_INDICATOR10,HMO_INDICATOR11,HMO_INDICATOR12,VALID_DATE_OF_DEATH_SWITCH,DATE_OF_DEATH,date_died,date_died_valid,year_of_death,most_common_physician_PRF_PHYSN_NPI,most_common_physician_n,most_common_physician_PRVDR_SPCLTY,most_common_primary_care_physician_PRF_PHYSN_NPI,most_common_primary_care_physician_n,most_common_primary_care_physician_PRVDR_SPCLTY,most_common_physician_in_facility_visits_count,most_common_physician_in_all_visits_count,most_common_physician_in_facility_non_exclusive_HCPCS_count,most_common_physician_in_all_non_exclusive_HCPCS_count,most_common_physician_in_facility_count,most_common_physician_in_all_count,most_common_physician_in_facility_visits_prp,most_common_physician_in_facility_non_exclusive_HCPCS_prp,most_common_physician_in_facility_prp,most_common_primary_care_physician_in_facility_visits_count,most_common_primary_care_physician_in_all_visits_count,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_all_non_exclusive_HCPCS_count,most_common_primary_care_physician_in_facility_count,most_common_primary_care_physician_in_all_count,most_common_primary_care_physician_in_facility_visits_prp,most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp,most_common_primary_care_physician_in_facility_prp,RUCC_2013,race,sex,age_group,urban,HMO_INDICATOR_sum,ENTITLEMENT_BUY_IN_IND_sum,months_alive,number_of_conditions,viclaims,vikeyword,tin1_nu,tin1_legal_name,tin2_nu,tin2_legal_name,name_last,name_first,name_middle,SPEC_PRIM_1_NAME,SPEC_PRIM_1,SPEC_BROAD,SEX,SPEC_PRIM_2,SPEC_PRIM_2_NAME,SPEC_SOURCE,SPEC_SOURCE_HOSP,POS_OFFICE,POS_INPAT,POS_OPD,POS_ER,POS_NURSING,POS_ASC,POS_RESID,POS_RETAIL,POS_URGENT,POS_OTHER,STATE,STATE_MULTI,CBSA_TYPE,CBSA_CD,CBSA_NAME,CBSA_MULTI,NPI_SRVC_LINES,NPI_ALLOWED_AMT,NPI_UNQ_BENES,TIN1_SRVC_MONTH,TIN1_SRVC_LINES,TIN1_ALLOWED_AMT,TIN1_UNQ_BENES,TIN2_SRVC_MONTH,TIN2_SRVC_LINES,TIN2_ALLOWED_AMT,TIN2_UNQ_BENES,BIRTH_YR,lns_office,lns_opd,most_common_primary_care_physician_integrated,most_common_primary_care_physician_integration_change,most_common_primary_care_physician_sum_integration_changes,most_common_primary_care_physician_integrated_once_and_stayed,most_common_primary_care_physician_never_integrated,most_common_primary_care_physician_year_integrated,most_common_primary_care_physician_years_from_integration,most_common_primary_care_physician_changed,last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,last_year_most_common_primary_care_physician_integrated,last_year_most_common_primary_care_physician_integration_change,last_year_most_common_primary_care_physician_sum_integration_changes,last_year_most_common_primary_care_physician_integrated_once_and_stayed,last_year_most_common_primary_care_physician_never_integrated,last_year_most_common_primary_care_physician_year_integrated,last_year_most_common_primary_care_physician_years_from_integration,hypertension_carrier,arthritis_carrier,IHD_carrier,diabetes_carrier,depression_carrier,hypertension,arthritis,IHD,diabetes,depression,exit_enter_status
<int>,<dbl>,<int>,<int>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr

In [57]:
comparator_exit_enter <- function(data) {
  require(tidyverse)
  require(dtplyr)


  result <- data %>%
    group_by(exit_enter_status) %>%
    summarise(
      n = n(),
      N_distinct_patients = length(unique(DESY_SORT_KEY)),
      N_distinct_specialist = length(unique(most_common_primary_care_physician_PRF_PHYSN_NPI)),
      Male = mean(sex == "Male", na.rm = T),
      Female = mean(sex == "Female", na.rm = T),
      Asian = mean(race == "Asian", na.rm = T),
      Black = mean(race == "Black", na.rm = T),
      Hispanic = mean(race == "Hispanic", na.rm = T),
      North_American_Native = mean(race == "North American Native", na.rm = T),
      White = mean(race == "White", na.rm = T),
      Other_race = mean(race == "Other", na.rm = T),
      avg_age = mean(AGE, na.rm = T),
      age_65_74 = mean(na.rm = T, age_group == "65-74"),
      age_75_84 = mean(na.rm = T, age_group == "75-84"),
      age_85_plus = mean(na.rm = T, age_group == "85+"),

      # West=mean(na.rm=T,census_region=="West"),
      # South=mean(na.rm=T,census_region=="South"),
      # Northeast=mean(na.rm=T,census_region=="Northeast"),
      # Midwest=mean(na.rm=T,census_region=="Midwest"),

      rural_prp = mean(RUCC_2013 > 3, na.rm = T),
      urban_prp = mean(RUCC_2013 <= 3, na.rm = T),
      hypertension = mean(hypertension, na.rm = T),
      arthritis = mean(arthritis, na.rm = T),
      IHD = mean(IHD, na.rm = T),
      diabetes = mean(diabetes, na.rm = T),
      depression = mean(depression, na.rm = T),
      two_conditions = mean(number_of_conditions == 2, na.rm = T),
      more_than_two_conditions = mean(number_of_conditions > 2, na.rm = T),
      died_the_year_of_integration = mean(year == year_of_death, na.rm = T)

      # primary_care_physician_integrated =
      #  sum(most_common_primary_care_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n(),
      # most_common_physician_integrated =
      #  sum(most_common_physician_in_facility_non_exclusive_HCPCS_prp >= integration_threshold, na.rm = T) / n()
    ) %>%
    as.data.table()

  return(result)
}

In [58]:
exit_enter_comparisons <- t(comparator_exit_enter(yearly_calculations))

In [59]:
exit_enter_comparisons

exit_enter_status,NA,Entering,Exiting,Remaining
n,9938958,18261,23686,31878
N_distinct_patients,2109378,17829,23269,31549
N_distinct_specialist,191213,7787,16807,6361
Male,0.4275588,0.4123542,0.4066537,0.4206349
Female,0.5724412,0.5876458,0.5933463,0.5793651
Asian,0.017861329,0.011664202,0.011356920,0.009159922
Black,0.07087705,0.07025902,0.06581947,0.06063743
Hispanic,0.01343501,0.01275943,0.01047032,0.00781103
North_American_Native,0.004488700,0.015552270,0.006121760,0.003827091
White,0.8632162,0.8664914,0.8822089,0.8962921


# Conditions in the dropping patients.

## In integrating doctors

In [60]:
exit_conditions <- function(data) {
  require(tidyverse)
  require(dtplyr)


  results_1 <- data %>%
    group_by(last_year_most_common_primary_care_physician_PRF_PHYSN_NPI) %>%
    summarise(
      n_exiting_patients = length(unique(.[exit_enter_status == "Exiting", DESY_SORT_KEY])),
      mortality = mean(.[exit_enter_status == "Exiting", year == year_of_death], na.rm = T),
      received_primary_care_elsewhere =
        mean(.[
          last_year_most_common_primary_care_physician_integration_change == "newly_integrated",
          !is.na(most_common_primary_care_physician_PRF_PHYSN_NPI)
        ], na.rm = T)
    ) %>%
    as.data.table()

  results_2 <- data %>%
    filter(most_common_primary_care_physician_years_from_integration == -1) %>%
    group_by(most_common_primary_care_physician_PRF_PHYSN_NPI) %>%
    summarise(n_patients_year_before_integration = length(unique(DESY_SORT_KEY))) %>%
    as.data.table()

  results <- inner_join(results_1, results_2, by = c("last_year_most_common_primary_care_physician_PRF_PHYSN_NPI" = "most_common_primary_care_physician_PRF_PHYSN_NPI")) %>%
    mutate(dropped_prp = n_exiting_patients / n_patients_year_before_integration) %>%
    as.data.table()

  result <- results %>%
    summarise(
      dropped_prp = mean(dropped_prp, na.rm = T),
      mortality_prp = mean(mortality, na.rm = T),
      received_primary_care_elsewhere = mean(received_primary_care_elsewhere, na.rm = T)
    ) %>%
    as.data.table()

  return(results)
}

In [61]:
results_exit_conditions_integrating <- exit_conditions(yearly_calculations)

In [62]:
head(results_exit_conditions_integrating)

last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,n_exiting_patients,mortality,received_primary_care_elsewhere,n_patients_year_before_integration,dropped_prp
<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>
1003007907,0,NaN,0.5000000,2,0.0000000
1003015314,0,NaN,0.0000000,3,0.0000000
1003037904,1,NaN,1.0000000,2,0.5000000
1003039678,1,NaN,0.5000000,2,0.5000000
1003042169,0,NaN,1.0000000,2,0.0000000
1003044694,9,0,0.8181818,11,0.8181818


## In independent docs

In [63]:
exit_conditions_all <- function(data) {
  require(tidyverse)
  require(dtplyr)

  results_1 <- data %>%
    filter(last_year_most_common_primary_care_physician_never_integrated == T) %>%
    group_by(last_year_most_common_primary_care_physician_PRF_PHYSN_NPI) %>%
    summarise(
      n_exiting_patients = length(unique(.[most_common_primary_care_physician_changed == TRUE, DESY_SORT_KEY])),
      mortality = mean(.[most_common_primary_care_physician_changed == TRUE, year == year_of_death], na.rm = T),
      received_primary_care_elsewhere =
        mean(!is.na(most_common_primary_care_physician_PRF_PHYSN_NPI), na.rm = T)
    ) %>%
    as.data.table()

  results_2 <- data %>%
    filter(most_common_primary_care_physician_never_integrated == T) %>%
    group_by(most_common_primary_care_physician_PRF_PHYSN_NPI) %>%
    summarise(n_patients_all_years = length(unique(DESY_SORT_KEY))) %>%
    as.data.table()

  results <- inner_join(results_1, results_2,
    by =
      c(
        "last_year_most_common_primary_care_physician_PRF_PHYSN_NPI" =
          "most_common_primary_care_physician_PRF_PHYSN_NPI"
      )
  ) %>%
    mutate(dropped_prp = n_exiting_patients / n_patients_all_years) %>%
    as.data.table()

  result <- results %>%
    summarise(
      dropped_prp = mean(dropped_prp, na.rm = T),
      mortality_prp = mean(mortality, na.rm = T),
      received_primary_care_elsewhere = mean(received_primary_care_elsewhere, na.rm = T)
    ) %>%
    as.data.table()

  return(results)
}

In [64]:
results_exit_conditions_all <- exit_conditions_all(yearly_calculations)

In [65]:
head(results_exit_conditions_all)

last_year_most_common_primary_care_physician_PRF_PHYSN_NPI,n_exiting_patients,mortality,received_primary_care_elsewhere,n_patients_all_years,dropped_prp
<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>
,7,NaN,0.8750000,10,0.7000000
1003000530,5,0,0.9000000,20,0.2500000
1003000902,3,0,0.8421053,11,0.2727273
1003000936,18,0,0.8923077,41,0.4390244
1003001884,0,NaN,0.7692308,5,0.0000000
1003002312,23,0,0.9111111,47,0.4893617


## Comparison test

In [66]:
t.test(results_exit_conditions_all$mortality, results_exit_conditions_integrating$mortality)


	Welch Two Sample t-test

data:  results_exit_conditions_all$mortality and results_exit_conditions_integrating$mortality
t = 3.2407, df = 2530.4, p-value = 0.001208
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.009567375 0.038884477
sample estimates:
mean of x mean of y 
0.2092254 0.1849994 


In [67]:
t.test(
  results_exit_conditions_all$received_primary_care_elsewhere,
  results_exit_conditions_integrating$received_primary_care_elsewhere
)


	Welch Two Sample t-test

data:  results_exit_conditions_all$received_primary_care_elsewhere and results_exit_conditions_integrating$received_primary_care_elsewhere
t = 18.149, df = 9340.4, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.05171325 0.06423640
sample estimates:
mean of x mean of y 
0.8497741 0.7917993 


In [68]:
t.test(
  results_exit_conditions_all$dropped_prp,
  results_exit_conditions_integrating$dropped_prp
)


	Welch Two Sample t-test

data:  results_exit_conditions_all$dropped_prp and results_exit_conditions_integrating$dropped_prp
t = 0.20924, df = 9600.6, p-value = 0.8343
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -0.006914043  0.008566495
sample estimates:
mean of x mean of y 
0.3674182 0.3665919 
